#### Why Snowpark?
The purpose of Snowpark lies in its ability to provide more flexibility, scalability, and integration for data processing and orchestration tasks.|

Orchestrating Jobs and Pipelines: We can automate Snowpark-based pipelines (UDFs, views, and other transformations) in a more flexible way. Snowpark code can be versioned and tested, making it easier to maintain and extend in your pipeline.

#### Note:
This notebook is to validate, clean and update data in tables of higher priority in terms of updating speed.

Data tables that need a faster updating speed will be processed here.

#### Desired Flow
1. Grab data from raw tables
2. Check for data, validate, check and update cleaned data onto the cleaned tables
3. Join tables via foreign keys and make hybrid master table
4. Create Schema Views

#### 1. Loading Raw Data In

Since we already push our data onto Snowflake, we can call for them in this notebook to run in Snowpark. This will be the first step to the data flow overview for establishing the CI/CD deployment & finalizing the ELT pipepline.

##### Tables in this notebook
- Sales.CustomerTransactions 
- Sales.CustomerCategories 
- Application.People 
- Purchasing.SupplierTransactions 
- Purchasing.PurchaseOrderLines 
- Warehouse.ColdRoomTemperatures 
- Warehouse.VehicleTemperatures
- Sales.Orders 
- Purchasing.PurchaseOrders
- Warehouse.StockItemTransactions
- Warehouse.StockItems
- Warehouse.StockItemHoldings
- Sales.Customers
- Sales.Invoices
- Sales.InvoiceLines
- Sales.Orderlines

In [ ]:
# RAW TABLES
# ---------------------------------------------------------------------------------------------
import time
from snowflake.snowpark import Session

# Define all tables organized by categories or schemas (CALL FOR RAW TABLES)
TABLE_DICT = {
    "application": {
        "schema": "KN_LOGISTICS.SNOWSQL", 
        "tables": [
            "APPLICATION_PEOPLE_RAW"
        ]
    },
    "purchasing": {
        "schema": "KN_LOGISTICS.SNOWSQL",
        "tables": [
            "PURCHASING_PURCHASEORDERLINES_RAW",
            "PURCHASING_PURCHASEORDERS_RAW"
        ]
    },
    "sales": {
        "schema": "KN_LOGISTICS.SNOWSQL",
        "tables": [
            "SALES_CUSTOMERCATEGORIES_RAW",
            "SALES_CUSTOMERS_RAW",
            "SALES_CUSTOMERTRANSACTIONS_RAW",
            "SALES_INVOICELINES_RAW",
            "SALES_INVOICES_RAW",
            "SALES_ORDERLINES_RAW",
            "SALES_ORDERS_RAW"
        ]
    },
    "warehouse": {
        "schema": "KN_LOGISTICS.SNOWSQL",
        "tables": [
            "WAREHOUSE_COLDROOMTEMPERATURES_RAW",
            "WAREHOUSE_STOCKITEMHOLDINGS_RAW",
            "WAREHOUSE_STOCKITEMS_RAW",
            "WAREHOUSE_STOCKITEMTRANSACTIONS_RAW",
            "WAREHOUSE_VEHICLETEMPERATURES_RAW"
        ]
    }
}

def load_raw_table(session, tname=None, schema=None):
    # Adjusted for direct use (no S3 staging assumed in your case)
    session.use_schema(schema)
    print(f"Loading table: {tname}")
    # If additional logic for transformations/loading is needed, add it here
    df = session.table(tname)
    df.show()  # Example action to verify table content

def load_all_tables(session):
    for category, data in TABLE_DICT.items():
        schema = data['schema']
        tables = data['tables']
        for tname in tables:
            load_raw_table(session, tname=tname, schema=schema)

def validate_tables(session):
    for category, data in TABLE_DICT.items():
        schema = data['schema']
        tables = data['tables']
        for tname in tables:
            session.use_schema(schema)
            print(f"Validating table: {tname}")
            print(f"Columns: {session.table(tname).columns}")

In [ ]:
# Add the utils package to our path and import the snowpark_utils function
import os, sys
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [ ]:
#from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
load_all_tables(session)

In [ ]:
validate_tables(session)

#### 2. Check for data, validate, check and update cleaned data onto the cleaned tables
After pulling in the raw data, we can check and validate these raw data to make sure they are of a certain format eligible to be pushed over to the cleaned tables. 

If not, we will clean the tables accordingly then update them over to the cleaned tables.

Some of the validation we can do is:
- Check for null values
- Check for duplicates in PK and UQ
- Check for invalid datetypes & text formats

After cleaning the data, we need to validate ONE MORE TIME to make sure the data has not been imported into the cleaned tables before. If validation succeeds, we update the current records over to the cleaned tables.
- Check if the data records exist in the cleaned tables (checkj if they are identitcal)

#### Cleaning APPLICATION_PEOPLE

In [ ]:
-- Create a clean table with the same structure as the raw table
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE AS
SELECT
  CAST(CASE WHEN PERSONID = 'NULL' THEN NULL ELSE PERSONID END AS VARCHAR(38)) AS PERSONID,
  CAST(CASE WHEN FULLNAME = 'NULL' THEN NULL ELSE FULLNAME END AS VARCHAR(50)) AS FULLNAME,
  CAST(CASE WHEN PREFERREDNAME = 'NULL' THEN NULL ELSE PREFERREDNAME END AS VARCHAR(50)) AS PREFERREDNAME,
  CAST(CASE WHEN SEARCHNAME = 'NULL' THEN NULL ELSE SEARCHNAME END AS VARCHAR(101)) AS SEARCHNAME,
  CAST(CASE WHEN ISEMPLOYEE = 'NULL' THEN NULL ELSE ISEMPLOYEE END AS VARCHAR(38)) AS ISEMPLOYEE,
  CAST(CASE WHEN ISSALESPERSON = 'NULL' THEN NULL ELSE ISSALESPERSON END AS VARCHAR(38)) AS ISSALESPERSON
FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE_RAW;

-- Verify the clean table
SELECT *
FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE;


---------------------------------------------
// data type conversion (to number): PERSONID
---------------------------------------------
// PERSONID
ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
ADD COLUMN PERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
SET PERSONID_NUM = TO_NUMBER(PERSONID);

SELECT PERSONID, PERSONID_NUM
FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
DROP COLUMN PERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
RENAME COLUMN PERSONID_NUM TO PERSONID;
// Check 10 rows of SUPPLIERID after updating the data type
SELECT PERSONID FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE LIMIT 10;

// Check 10 rows of the whole table after updating the data type
SELECT * FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE;

---------------------------------------------
-- Data type conversion (BIT to BOOLEAN)
---------------------------------------------
-- Convert ISEMPLOYEE to BOOLEAN
ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
ADD COLUMN ISEMPLOYEE_BOOL BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
SET ISEMPLOYEE_BOOL = CASE 
    WHEN ISEMPLOYEE = '1' THEN TRUE
    WHEN ISEMPLOYEE = '0' THEN FALSE
    ELSE NULL
END;

SELECT ISEMPLOYEE, ISEMPLOYEE_BOOL
FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
DROP COLUMN ISEMPLOYEE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
RENAME COLUMN ISEMPLOYEE_BOOL TO ISEMPLOYEE;

---------------------------------------------
-- Convert ISSALESPERSON to BOOLEAN
---------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
ADD COLUMN ISSALESPERSON_BOOL BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
SET ISSALESPERSON_BOOL = CASE 
    WHEN ISSALESPERSON = '1' THEN TRUE
    WHEN ISSALESPERSON = '0' THEN FALSE
    ELSE NULL
END;

SELECT ISSALESPERSON, ISSALESPERSON_BOOL
FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
DROP COLUMN ISSALESPERSON;

ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
RENAME COLUMN ISSALESPERSON_BOOL TO ISSALESPERSON;


SELECT ISSALESPERSON,ISEMPLOYEE FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE LIMIT 10;

---------------------------------------
--Adding of primary key to table
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
ADD CONSTRAINT PK_APPLICATION_PEOPLE_PERSONID
PRIMARY KEY (PERSONID);


-------------------------------------------------------
-- ERROR HANDLING
-------------------------------------------------------
WITH CTE AS (
    SELECT 
        PERSONID,
        FULLNAME,
        PREFERREDNAME,
        SEARCHNAME,
        ISEMPLOYEE,
        ISSALESPERSON,
        LAG(PERSONID) OVER (ORDER BY PERSONID) AS prev_personid,
        ROW_NUMBER() OVER (ORDER BY PERSONID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE
)
SELECT 
    FULLNAME,
    PREFERREDNAME,
    SEARCHNAME,
    ISEMPLOYEE,
    ISSALESPERSON,
    CASE
        WHEN PERSONID IS NULL THEN prev_personid + 1
        ELSE PERSONID
    END AS PERSONID
FROM CTE
ORDER BY row_num;



-------------------------------------------------------
--Adding of foreign key to table
-------------------------------------------------------
-- THERE ARE NO FOREIGN KEYS FOR THIS TABLE

#### Cleaning PURCHASING_SUPPLIERTRANSACTIONS

In [ ]:
-- Create a clean table with the same structure as the raw table but we pull data from the raw table into this table (final table)
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS AS
SELECT
  CAST(CASE WHEN SUPPLIERTRANSACTIONID = 'NULL' THEN NULL ELSE SUPPLIERTRANSACTIONID END AS VARCHAR(38)) AS SUPPLIERTRANSACTIONID,
  CAST(CASE WHEN SUPPLIERID = 'NULL' THEN NULL ELSE SUPPLIERID END AS VARCHAR(38)) AS SUPPLIERID,
  CAST(CASE WHEN TRANSACTIONTYPEID = 'NULL' THEN NULL ELSE TRANSACTIONTYPEID END AS VARCHAR(38)) AS TRANSACTIONTYPEID,
  CAST(CASE WHEN PURCHASEORDERID = 'NULL' THEN NULL ELSE PURCHASEORDERID END AS VARCHAR(38)) AS PURCHASEORDERID,
  CAST(CASE WHEN PAYMENTMETHODID = 'NULL' THEN NULL ELSE PAYMENTMETHODID END AS VARCHAR(38)) AS PAYMENTMETHODID,
  CAST(CASE WHEN SUPPLIERINVOICENUMBER = 'NULL' THEN NULL ELSE SUPPLIERINVOICENUMBER END AS VARCHAR(20)) AS SUPPLIERINVOICENUMBER,
  CAST(CASE WHEN TRANSACTIONDATE = 'NULL' THEN NULL ELSE TRANSACTIONDATE END AS VARCHAR(20)) AS TRANSACTIONDATE,
  CAST(CASE WHEN AMOUNTEXCLUDINGTAX = 'NULL' THEN NULL ELSE AMOUNTEXCLUDINGTAX END AS VARCHAR(38)) AS AMOUNTEXCLUDINGTAX,
  CAST(CASE WHEN TAXAMOUNT = 'NULL' THEN NULL ELSE TAXAMOUNT END AS VARCHAR(38)) AS TAXAMOUNT,
  CAST(CASE WHEN TRANSACTIONAMOUNT = 'NULL' THEN NULL ELSE TRANSACTIONAMOUNT END AS VARCHAR(38)) AS TRANSACTIONAMOUNT,
  CAST(CASE WHEN OUTSTANDINGBALANCE = 'NULL' THEN NULL ELSE OUTSTANDINGBALANCE END AS VARCHAR(38)) AS OUTSTANDINGBALANCE,
  CAST(CASE WHEN FINALIZATIONDATE = 'NULL' THEN NULL ELSE FINALIZATIONDATE END AS VARCHAR(20)) AS FINALIZATIONDATE,
  CAST(CASE WHEN ISFINALIZED = 'NULL' THEN NULL ELSE ISFINALIZED END AS VARCHAR(38)) AS ISFINALIZED
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS_RAW;

-- check for 'null' for VARCHAR DATA TYPE COLUMNS ONLY
SELECT
    SUM(CASE WHEN SUPPLIERID IS NULL THEN 1 END) AS NULL_COUNT_SUPPLIERID,
    SUM(CASE WHEN PURCHASEORDERID IS NULL THEN 1 END) AS NULL_COUNT_PURCHASEORDERID,
    SUM(CASE WHEN PAYMENTMETHODID IS NULL THEN 1 END) AS NULL_COUNT_PAYMENTMETHODID,
    SUM(CASE WHEN SUPPLIERINVOICENUMBER IS NULL THEN 1 END) AS NULL_COUNT_SUPPLIERINVOICENUMBER,
    SUM(CASE WHEN TRANSACTIONDATE IS NULL THEN 1 END) AS NULL_COUNT_TRANSACTIONDATE,
    SUM(CASE WHEN FINALIZATIONDATE IS NULL THEN 1 END) AS NULL_COUNT_FINALIZATIONDATE,
FROM 
    KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

--  check for actual null values
SELECT 
    COUNT(CASE WHEN SUPPLIERTRANSACTIONID IS NULL THEN 1 END) AS count_SUPPLIERTRANSACTIONID_NULL,
    COUNT(CASE WHEN SUPPLIERID IS NULL THEN 1 END) AS count_SUPPLIERID_NULL,
    COUNT(CASE WHEN TRANSACTIONTYPEID IS NULL THEN 1 END) AS count_TRANSACTIONTYPEID_NULL,
    COUNT(CASE WHEN PURCHASEORDERID IS NULL THEN 1 END) AS count_PURCHASEORDERID_NULL,
    COUNT(CASE WHEN PAYMENTMETHODID IS NULL THEN 1 END) AS count_PAYMENTMETHODID_NULL,
    COUNT(CASE WHEN SUPPLIERINVOICENUMBER IS NULL THEN 1 END) AS count_SUPPLIERINVOICENUMBER_NULL,
    COUNT(CASE WHEN TRANSACTIONDATE IS NULL THEN 1 END) AS count_TRANSACTIONDATE_NULL,
    COUNT(CASE WHEN AMOUNTEXCLUDINGTAX IS NULL THEN 1 END) AS count_AMOUNTEXCLUDINGTAX_NULL,
    COUNT(CASE WHEN TAXAMOUNT IS NULL THEN 1 END) AS count_TAXAMOUNT_NULL,
    COUNT(CASE WHEN TRANSACTIONAMOUNT IS NULL THEN 1 END) AS count_TRANSACTIONAMOUNT_NULL,
    COUNT(CASE WHEN OUTSTANDINGBALANCE IS NULL THEN 1 END) AS count_OUTSTANDINGBALANCE_NULL,
    COUNT(CASE WHEN FINALIZATIONDATE IS NULL THEN 1 END) AS count_FINALIZATIONDATE_NULL,
    COUNT(CASE WHEN ISFINALIZED IS NULL THEN 1 END) AS count_ISFINALIZED_NULL
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

---------------------------------------------
-- data type conversion (to number)
---------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN SUPPLIERTRANSACTIONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET SUPPLIERTRANSACTIONID_NUM = TO_NUMBER(PURCHASEORDERID);

SELECT SUPPLIERTRANSACTIONID, SUPPLIERTRANSACTIONID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN SUPPLIERTRANSACTIONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN SUPPLIERTRANSACTIONID_NUM TO SUPPLIERTRANSACTIONID;

---------------------------------------------
-- data type conversion (to number)
---------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN TRANSACTIONTYPEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET TRANSACTIONTYPEID_NUM = TO_NUMBER(TRANSACTIONTYPEID);

SELECT TRANSACTIONTYPEID, TRANSACTIONTYPEID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN TRANSACTIONTYPEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN TRANSACTIONTYPEID_NUM TO TRANSACTIONTYPEID;

---------------------------------------------
-- data type conversion (to number)
---------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN PAYMENTMETHODID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET PAYMENTMETHODID_NUM = TO_NUMBER(PAYMENTMETHODID);

SELECT PAYMENTMETHODID, PAYMENTMETHODID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN PAYMENTMETHODID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN PAYMENTMETHODID_NUM TO PAYMENTMETHODID;


---------------------------------------------
-- data type conversion (to number)
---------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN PURCHASEORDERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET PURCHASEORDERID_NUM = TO_NUMBER(PURCHASEORDERID);

SELECT PURCHASEORDERID, PURCHASEORDERID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN PURCHASEORDERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN PURCHASEORDERID_NUM TO PURCHASEORDERID;

---------------------------------------------
-- data type conversion (to number)
---------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN SUPPLIERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET SUPPLIERID_NUM = TO_NUMBER(SUPPLIERID);

SELECT SUPPLIERID, SUPPLIERID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN SUPPLIERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN SUPPLIERID_NUM TO SUPPLIERID;

--------------------------------------------------
-- data type conversion (to float) AMOUNTEXCLUDINGTAX 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN AMOUNTEXCLUDINGTAX_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET AMOUNTEXCLUDINGTAX_NUM = TO_DECIMAL(AMOUNTEXCLUDINGTAX, 18, 2);

SELECT AMOUNTEXCLUDINGTAX, AMOUNTEXCLUDINGTAX_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN AMOUNTEXCLUDINGTAX;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN AMOUNTEXCLUDINGTAX_NUM TO AMOUNTEXCLUDINGTAX;

--------------------------------------------------
-- data type conversion (to float) TAXAMOUNT 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN TAXAMOUNT_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET TAXAMOUNT_NUM = TO_DECIMAL(TAXAMOUNT, 18, 2);

SELECT TAXAMOUNT, TAXAMOUNT_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN TAXAMOUNT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN TAXAMOUNT_NUM TO TAXAMOUNT;

--------------------------------------------------
-- data type conversion (to float) TRANSACTIONAMOUNT 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN TRANSACTIONAMOUNT_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET TRANSACTIONAMOUNT_NUM = TO_DECIMAL(TRANSACTIONAMOUNT, 18, 2);

SELECT TRANSACTIONAMOUNT, TRANSACTIONAMOUNT_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN TRANSACTIONAMOUNT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN TRANSACTIONAMOUNT_NUM TO TRANSACTIONAMOUNT;

--------------------------------------------------
-- data type conversion (to float) OUTSTANDINGBALANCE 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN OUTSTANDINGBALANCE_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET OUTSTANDINGBALANCE_NUM = TO_DECIMAL(OUTSTANDINGBALANCE, 18, 2);

SELECT OUTSTANDINGBALANCE, OUTSTANDINGBALANCE_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN OUTSTANDINGBALANCE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN OUTSTANDINGBALANCE_NUM TO OUTSTANDINGBALANCE;

--------------------------------------------------
-- data type conversion (to date) FINALIZATIONDATE 
--------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN FINALIZATIONDATE_DATE DATE;

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET FINALIZATIONDATE_DATE = TO_DATE(FINALIZATIONDATE, 'DD/MM/YYYY');

SELECT FINALIZATIONDATE, FINALIZATIONDATE_DATE
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN FINALIZATIONDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN FINALIZATIONDATE_DATE TO FINALIZATIONDATE;


--------------------------------------------------
-- data type conversion (to date) TRANSACTIONDATE
--------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN TRANSACTIONDATE_DATE DATE;

-- Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET TRANSACTIONDATE_DATE = TO_DATE(TRANSACTIONDATE, 'DD/MM/YYYY');

SELECT TRANSACTIONDATE, TRANSACTIONDATE_DATE
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN TRANSACTIONDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN TRANSACTIONDATE_DATE TO TRANSACTIONDATE;

--------------------------------------------------
--- data type conversion (to float) ISFINALIZED 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD COLUMN ISFINALIZED_NUM BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
SET ISFINALIZED_NUM = TO_BOOLEAN(ISFINALIZED);

SELECT ISFINALIZED, ISFINALIZED_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
DROP COLUMN ISFINALIZED;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
RENAME COLUMN ISFINALIZED_NUM TO ISFINALIZED;

-- primary key
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD CONSTRAINT PK_PURCHASING_SUPPLIERTRANSACTIONS_SUPPLIERTRANSACTIONID
PRIMARY KEY (SUPPLIERTRANSACTIONID);

-- null error handling for primary key (increment 1 to impute null value primary keys from the previous record)
WITH CTE AS (
    SELECT
        SUPPLIERID,
        SUPPLIERINVOICENUMBER,
        PURCHASEORDERID,
        AMOUNTEXCLUDINGTAX,
        TAXAMOUNT,
        TRANSACTIONAMOUNT,
        OUTSTANDINGBALANCE,
        FINALIZATIONDATE,
        TRANSACTIONDATE,
        ISFINALIZED,
        SUPPLIERTRANSACTIONID,
        TRANSACTIONTYPEID,
        PAYMENTMETHODID,
        LAG(SUPPLIERTRANSACTIONID) OVER (ORDER BY SUPPLIERTRANSACTIONID) AS prev_SUPPLIERTRANSACTIONID,
        ROW_NUMBER() OVER (ORDER BY SUPPLIERTRANSACTIONID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
)
SELECT 
    SUPPLIERINVOICENUMBER,
    PURCHASEORDERID,
    SUPPLIERID,
    AMOUNTEXCLUDINGTAX,
    TAXAMOUNT,
    TRANSACTIONAMOUNT,
    OUTSTANDINGBALANCE,
    FINALIZATIONDATE,
    TRANSACTIONDATE,
    ISFINALIZED,
    TRANSACTIONTYPEID,
    PAYMENTMETHODID,
    CASE
        WHEN SUPPLIERTRANSACTIONID IS NULL THEN prev_SUPPLIERTRANSACTIONID + 1
        ELSE SUPPLIERTRANSACTIONID
    END AS SUPPLIERTRANSACTIONID
FROM CTE
ORDER BY row_num;

-- Add the foreign key constraint for SUPPLIERID
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD CONSTRAINT FK_PURCHASING_SUPPLIERTRANSACTIONS_SUPPLIERID 
FOREIGN KEY (SUPPLIERID) 
REFERENCES KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERS(SUPPLIERID);

-- Add the foreign key constraint for TRANSACTIONTYPEID
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD CONSTRAINT FK_PURCHASING_SUPPLIERTRANSACTIONS_TRANSACTIONTYPEID 
FOREIGN KEY (TRANSACTIONTYPEID) 
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_TRANSACTIONTYPES(TRANSACTIONTYPEID);

-- Add the foreign key constraint for PURCHASEORDERID
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD CONSTRAINT FK_PURCHASING_SUPPLIERTRANSACTIONS_PURCHASEORDERID 
FOREIGN KEY (PURCHASEORDERID) 
REFERENCES KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS(PURCHASEORDERID);

-- Add the foreign key constraint for PAYMENTMETHODID
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERTRANSACTIONS
ADD CONSTRAINT FK_PURCHASING_SUPPLIERTRANSACTIONS_PAYMENTMETHODID 
FOREIGN KEY (PAYMENTMETHODID) 
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PAYMENTMETHODS(PAYMENTMETHODID);

#### Cleaning PURCHASING_PURCHASEORDERS

In [ ]:
-- Create a clean table with the same structure as the raw table but we pull data from the raw table into this table (final table)
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS AS
SELECT
  CAST(CASE WHEN PURCHASEORDERID = 'NULL' THEN NULL ELSE PURCHASEORDERID END AS VARCHAR(38)) AS PURCHASEORDERID,
  CAST(CASE WHEN SUPPLIERID = 'NULL' THEN NULL ELSE SUPPLIERID END AS VARCHAR(38)) AS SUPPLIERID,
  CAST(CASE WHEN ORDERDATE = 'NULL' THEN NULL ELSE ORDERDATE END AS VARCHAR(20)) AS ORDERDATE,
  CAST(CASE WHEN DELIVERYMETHODID = 'NULL' THEN NULL ELSE DELIVERYMETHODID END AS VARCHAR(38)) AS DELIVERYMETHODID,
  CAST(CASE WHEN CONTACTPERSONID = 'NULL' THEN NULL ELSE CONTACTPERSONID END AS VARCHAR(38)) AS CONTACTPERSONID,
  CAST(CASE WHEN EXPECTEDDELIVERYDATE = 'NULL' THEN NULL ELSE EXPECTEDDELIVERYDATE END AS VARCHAR(20)) AS EXPECTEDDELIVERYDATE,
  CAST(CASE WHEN SUPPLIERREFERENCE = 'NULL' THEN NULL ELSE SUPPLIERREFERENCE END AS VARCHAR(20)) AS SUPPLIERREFERENCE,
  CAST(CASE WHEN ISORDERFINALIZED = 'NULL' THEN NULL ELSE ISORDERFINALIZED END AS VARCHAR(38)) AS ISORDERFINALIZED
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS_RAW;

-- Total Number of Null values for all columns
SELECT COUNT(*) FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
WHERE PURCHASEORDERID IS NULL
OR SUPPLIERID IS NULL
OR ORDERDATE IS NULL
OR DELIVERYMETHODID IS NULL
OR CONTACTPERSONID IS NULL
OR EXPECTEDDELIVERYDATE IS NULL
OR SUPPLIERREFERENCE IS NULL
OR ISORDERFINALIZED IS NULL;

-- Null value checks for all columns (only varchar data type columns)
SELECT 
    SUM(CASE WHEN SUPPLIERREFERENCE = 'NULL' THEN 1 ELSE 0 END) AS NULL_COUNT_SUPPLIERREFERENCE
FROM 
    KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;
    

-- Null value checks for all columns (only numeric/boolean data type columns)
SELECT 
    COUNT(CASE WHEN PURCHASEORDERID IS NULL THEN 1 END) AS count_PURCHASEORDERID_NULL,
    COUNT(CASE WHEN SUPPLIERID IS NULL THEN 1 END) AS count_SUPPLIERID_NULL,
    COUNT(CASE WHEN ORDERDATE IS NULL THEN 1 END) AS count_ORDERDATE_NULL,
    COUNT(CASE WHEN DELIVERYMETHODID IS NULL THEN 1 END) AS count_DELIVERYMETHODID_NULL,
    COUNT(CASE WHEN CONTACTPERSONID IS NULL THEN 1 END) AS count_CONTACTPERSONID_NULL,
    COUNT(CASE WHEN EXPECTEDDELIVERYDATE IS NULL THEN 1 END) AS count_EXPECTEDDELIVERYDATE_NULL,
    COUNT(CASE WHEN SUPPLIERREFERENCE IS NULL THEN 1 END) AS count_SUPPLIERREFERENCE_NULL,
    COUNT(CASE WHEN ISORDERFINALIZED IS NULL THEN 1 END) AS count_ISORDERFINALIZED_NULL
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;

--------------------------------------------------
-- data type conversion (to date) ORDERDATE 
--------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD COLUMN ORDERDATE_DATE DATE;

-- Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
SET ORDERDATE_DATE = TO_DATE(ORDERDATE, 'DD/MM/YYYY');

SELECT ORDERDATE, ORDERDATE_DATE
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
DROP COLUMN ORDERDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
RENAME COLUMN ORDERDATE_DATE TO ORDERDATE;

--------------------------------------------------
-- data type conversion (to date) EXPECTEDDELIVERYDATE 
--------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD COLUMN EXPECTEDDELIVERYDATE_DATE DATE;

-- Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
SET EXPECTEDDELIVERYDATE_DATE = TO_DATE(EXPECTEDDELIVERYDATE, 'DD/MM/YYYY');

SELECT EXPECTEDDELIVERYDATE, EXPECTEDDELIVERYDATE_DATE
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
DROP COLUMN EXPECTEDDELIVERYDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
RENAME COLUMN EXPECTEDDELIVERYDATE_DATE TO EXPECTEDDELIVERYDATE;

--------------------------------------------------
-- data type conversion (to number) SUPPLIERID 
--------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD COLUMN SUPPLIERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
SET SUPPLIERID_NUM = TO_NUMBER(SUPPLIERID);

SELECT SUPPLIERID, SUPPLIERID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
DROP COLUMN SUPPLIERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
RENAME COLUMN SUPPLIERID_NUM TO SUPPLIERID;

SELECT *
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
WHERE SUPPLIERID = 7;

--check the number of rows of SUPPLIERID = 7(random SUPPLIERID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
WHERE SUPPLIERID = 7;

--------------------------------------------------
-- data type conversion (to number) PURCHASEORDERID 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD COLUMN PURCHASEORDERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
SET PURCHASEORDERID_NUM = TO_NUMBER(PURCHASEORDERID);

SELECT PURCHASEORDERID, PURCHASEORDERID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
DROP COLUMN PURCHASEORDERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
RENAME COLUMN PURCHASEORDERID_NUM TO PURCHASEORDERID;

--check the number of rows of PURCHASEORDERID = 109(random PURCHASEORDERID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
WHERE PURCHASEORDERID = 109;

--------------------------------------------------
-- data type conversion (to number) CONTACTPERSONID 
--------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD COLUMN CONTACTPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
SET CONTACTPERSONID_NUM = TO_NUMBER(CONTACTPERSONID);

SELECT CONTACTPERSONID, CONTACTPERSONID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
DROP COLUMN CONTACTPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
RENAME COLUMN CONTACTPERSONID_NUM TO CONTACTPERSONID;

--check the number of rows of CONTACTPERSONID = 2(random CONTACTPERSONID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
WHERE CONTACTPERSONID = 2;

--------------------------------------------------
-- data type conversion (to number) DELIVERYMETHODID 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD COLUMN DELIVERYMETHODID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
SET DELIVERYMETHODID_NUM = TO_NUMBER(DELIVERYMETHODID);

SELECT DELIVERYMETHODID, DELIVERYMETHODID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
DROP COLUMN DELIVERYMETHODID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
RENAME COLUMN DELIVERYMETHODID_NUM TO DELIVERYMETHODID;

--check the number of rows of DELIVERYMETHODID = 1(random DELIVERYMETHODID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
WHERE DELIVERYMETHODID = 2;

--------------------------------------------------
-- data type conversion (to Boolean) ISORDERLINEFINALIZED 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD COLUMN ISORDERLINEFINALIZED_NUM BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
SET ISORDERLINEFINALIZED_NUM = TO_BOOLEAN(ISORDERFINALIZED);

SELECT ISORDERFINALIZED, ISORDERLINEFINALIZED_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
DROP COLUMN ISORDERFINALIZED;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
RENAME COLUMN ISORDERLINEFINALIZED_NUM TO ISORDERFINALIZED;

SELECT * FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS LIMIT 10;

-- primary key
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD CONSTRAINT PK_PURCHASING_PURCHASEORDERS_PURCHASEORDERID
PRIMARY KEY (PURCHASEORDERID);

-- null error handling for primary key (increment 1 to impute null value primary keys from the previous record)
WITH CTE AS (
    SELECT 
        PurchaseOrderID,
        SupplierID,
        OrderDate,
        DeliveryMethodID,
        ContactPersonID,
        ExpectedDeliveryDate,
        SupplierReference,
        IsOrderFinalized,
        LAG(PurchaseOrderID) OVER (ORDER BY PurchaseOrderID) AS prev_PurchaseOrderID,
        ROW_NUMBER() OVER (ORDER BY PurchaseOrderID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
)
SELECT 
        SupplierID,
        OrderDate,
        DeliveryMethodID,
        ContactPersonID,
        ExpectedDeliveryDate,
        SupplierReference,
        IsOrderFinalized,
    CASE
        WHEN PurchaseOrderID IS NULL THEN prev_PurchaseOrderID + 1
        ELSE PurchaseOrderID
    END AS PurchaseOrderID
FROM CTE
ORDER BY row_num;


-- foreign keys
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD CONSTRAINT FK_Purchasing_PurchaseOrders_DeliveryMethodID_Application_DeliveryMethods
FOREIGN KEY (DeliveryMethodID)
REFERENCES KN_LOGISTICS.SNOWSQL.Application_DeliveryMethods(DeliveryMethodID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD CONSTRAINT FK_Purchasing_PurchaseOrders_ContactPersonID_Application_People
FOREIGN KEY (ContactPersonID)
REFERENCES KN_LOGISTICS.SNOWSQL.Application_People(PersonID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS
ADD CONSTRAINT FK_Purchasing_PurchaseOrders_SupplierID_Purchasing_Suppliers
FOREIGN KEY (SupplierID)
REFERENCES KN_LOGISTICS.SNOWSQL.Purchasing_Suppliers(SupplierID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.Purchasing_PurchaseOrderLines
ADD CONSTRAINT FK_Purchasing_PurchaseOrderLines_PurchaseOrderID_Purchasing_PurchaseOrders
FOREIGN KEY (PurchaseOrderID)
REFERENCES KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS(PurchaseOrderID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD CONSTRAINT FK_Warehouse_StockItemTransactions_PurchaseOrderID_Purchasing_PurchaseOrders
FOREIGN KEY (PURCHASEORDERID)
REFERENCES KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS(PURCHASEORDERID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.Purchasing_SupplierTransactions
ADD CONSTRAINT FK_Purchasing_SupplierTransactions_PurchaseOrderID_Purchasing_PurchaseOrders
FOREIGN KEY (PurchaseOrderID)
REFERENCES KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS(PurchaseOrderID);

#### Cleaning PURCHASING_PURCHASEORDERLINES

In [ ]:
-- Create a clean table with the same structure as the raw table but we pull data from the raw table into this table (final table)
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES AS
SELECT
  CAST(CASE WHEN PURCHASEORDERLINEID = 'NULL' THEN NULL ELSE PURCHASEORDERLINEID END AS VARCHAR(38)) AS PURCHASEORDERLINEID,
  CAST(CASE WHEN PURCHASEORDERID = 'NULL' THEN NULL ELSE PURCHASEORDERID END AS VARCHAR(38)) AS PURCHASEORDERID,
  CAST(CASE WHEN STOCKITEMID = 'NULL' THEN NULL ELSE STOCKITEMID END AS VARCHAR(38)) AS STOCKITEMID,
  CAST(CASE WHEN ORDEREDOUTERS = 'NULL' THEN NULL ELSE ORDEREDOUTERS END AS VARCHAR(38)) AS ORDEREDOUTERS,
  CAST(CASE WHEN DESCRIPTION = 'NULL' THEN NULL ELSE DESCRIPTION END AS VARCHAR(100)) AS DESCRIPTION,
  CAST(CASE WHEN RECEIVEDOUTERS = 'NULL' THEN NULL ELSE RECEIVEDOUTERS END AS VARCHAR(38)) AS RECEIVEDOUTERS,
  CAST(CASE WHEN PACKAGETYPEID = 'NULL' THEN NULL ELSE PACKAGETYPEID END AS VARCHAR(38)) AS PACKAGETYPEID,
  CAST(CASE WHEN EXPECTEDUNITPRICEPEROUTER = 'NULL' THEN NULL ELSE EXPECTEDUNITPRICEPEROUTER END AS VARCHAR(38)) AS EXPECTEDUNITPRICEPEROUTER,
  CAST(CASE WHEN LASTRECEIPTDATE = 'NULL' THEN NULL ELSE LASTRECEIPTDATE END AS VARCHAR(20)) AS LASTRECEIPTDATE,
  CAST(CASE WHEN ISORDERLINEFINALIZED = 'NULL' THEN NULL ELSE ISORDERLINEFINALIZED END AS VARCHAR(38)) AS ISORDERLINEFINALIZED
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES_RAW;

--------------------------------------------------
-- data type conversion (to float) EXPECTEDUNITPRICEPEROUTER 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN ISORDERLINEFINALIZED_NUM BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET ISORDERLINEFINALIZED_NUM = TO_BOOLEAN(ISORDERLINEFINALIZED);

SELECT ISORDERLINEFINALIZED, ISORDERLINEFINALIZED_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN ISORDERLINEFINALIZED;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN ISORDERLINEFINALIZED_NUM TO ISORDERLINEFINALIZED;

--------------------------------------------------
-- data type conversion (to float) EXPECTEDUNITPRICEPEROUTER 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN EXPECTEDUNITPRICEPEROUTER_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET EXPECTEDUNITPRICEPEROUTER_NUM = TO_DECIMAL(EXPECTEDUNITPRICEPEROUTER, 18, 2);

SELECT EXPECTEDUNITPRICEPEROUTER, EXPECTEDUNITPRICEPEROUTER_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN EXPECTEDUNITPRICEPEROUTER;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN EXPECTEDUNITPRICEPEROUTER_NUM TO EXPECTEDUNITPRICEPEROUTER;

--------------------------------------------------
-- data type conversion (to number) PACKAGETYPEID 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN PACKAGETYPEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET PACKAGETYPEID_NUM = TO_NUMBER(PACKAGETYPEID);

SELECT PACKAGETYPEID, PACKAGETYPEID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN PACKAGETYPEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN PACKAGETYPEID_NUM TO PACKAGETYPEID;

--------------------------------------------------
-- data type conversion (to number) RECEIVEDOUTERS 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN RECEIVEDOUTERS_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET RECEIVEDOUTERS_NUM = TO_NUMBER(RECEIVEDOUTERS);

SELECT RECEIVEDOUTERS, RECEIVEDOUTERS_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN RECEIVEDOUTERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN RECEIVEDOUTERS_NUM TO RECEIVEDOUTERS;

--------------------------------------------------
-- data type conversion (to number) ORDEREDOUTERS 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN ORDEREDOUTERS_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET ORDEREDOUTERS_NUM = TO_NUMBER(ORDEREDOUTERS);

SELECT ORDEREDOUTERS, ORDEREDOUTERS_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN ORDEREDOUTERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN ORDEREDOUTERS_NUM TO ORDEREDOUTERS;

--------------------------------------------------
-- data type conversion (to number) STOCKITEMID 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN STOCKITEMID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET STOCKITEMID_NUM = TO_NUMBER(STOCKITEMID);

SELECT STOCKITEMID, STOCKITEMID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN STOCKITEMID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN STOCKITEMID_NUM TO STOCKITEMID;

--------------------------------------------------
-- data type conversion (to number) PURCHASEORDERLINEID 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN PURCHASEORDERLINEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET PURCHASEORDERLINEID_NUM = TO_NUMBER(PURCHASEORDERLINEID);

SELECT PURCHASEORDERLINEID, PURCHASEORDERLINEID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN PURCHASEORDERLINEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN PURCHASEORDERLINEID_NUM TO PURCHASEORDERLINEID;

--------------------------------------------------
-- data type conversion (to number) PURCHASEORDERID 
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN PURCHASEORDERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET PURCHASEORDERID_NUM = TO_NUMBER(PURCHASEORDERID);

SELECT PURCHASEORDERID, PURCHASEORDERID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN PURCHASEORDERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN PURCHASEORDERID_NUM TO PURCHASEORDERID;

--------------------------------------------------
-- data type conversion (to date) LASTRECEIPTDATE 
--------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD COLUMN LASTRECEIPTDATE_DATE DATE;

-- Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
SET LASTRECEIPTDATE_DATE = TO_DATE(LASTRECEIPTDATE, 'DD/MM/YYYY');

SELECT LASTRECEIPTDATE, LASTRECEIPTDATE_DATE
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
DROP COLUMN LASTRECEIPTDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
RENAME COLUMN LASTRECEIPTDATE_DATE TO LASTRECEIPTDATE;

SELECT * FROM KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES LIMIT 10;


--primary key
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD CONSTRAINT PK_PURCHASING_PURCHASEORDERLINES_PURCHASEORDERLINEID
PRIMARY KEY (PURCHASEORDERLINEID);


-- null error handling for primary key (increment 1 to impute null value primary keys from the previous record)
WITH CTE AS (
    SELECT 
        PurchaseOrderLineID,
        PurchaseOrderID,
        StockItemID,
        OrderedOuters,
        Description,
        ReceivedOuters,
        PackageTypeID,
        ExpectedUnitPricePerOuter,
        LastReceiptDate,
        IsOrderLineFinalized,
        LAG(PurchaseOrderLineID) OVER (ORDER BY PurchaseOrderLineID) AS prev_PurchaseOrderLineID,
        ROW_NUMBER() OVER (ORDER BY PurchaseOrderLineID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.Purchasing_PurchaseOrderLines
)
SELECT 
    PurchaseOrderID,
    StockItemID,
    OrderedOuters,
    Description,
    ReceivedOuters,
    PackageTypeID,
    ExpectedUnitPricePerOuter,
    LastReceiptDate,
    IsOrderLineFinalized,
    CASE
        WHEN PurchaseOrderLineID IS NULL THEN prev_PurchaseOrderLineID + 1
        ELSE PurchaseOrderLineID
    END AS PurchaseOrderLineID
FROM CTE
ORDER BY row_num;


-- Adding the foreign keys
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD CONSTRAINT FK_PURCHASEORDERLINES_PURCHASEORDERID
FOREIGN KEY (PURCHASEORDERID)
REFERENCES KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERS (PURCHASEORDERID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD CONSTRAINT FK_PURCHASEORDERLINES_STOCKITEMID
FOREIGN KEY (STOCKITEMID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS (STOCKITEMID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_PURCHASEORDERLINES
ADD CONSTRAINT FK_PURCHASEORDERLINES_PACKAGETYPEID
FOREIGN KEY (PACKAGETYPEID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_PACKAGETYPES(PACKAGETYPEID);

#### Cleaning SALES_CUSTOMERCATEGORIES

In [ ]:
-- Create a clean table with the same structure as the raw table
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES AS
SELECT
  CAST(CASE WHEN SUPPLIERCATEGORYID = 'NULL' THEN NULL ELSE SUPPLIERCATEGORYID END AS VARCHAR(38)) AS SUPPLIERCATEGORYID,
  CAST(CASE WHEN SUPPLIERCATEGORYNAME = 'NULL' THEN NULL ELSE SUPPLIERCATEGORYNAME END AS VARCHAR(50)) AS SUPPLIERCATEGORYNAME
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES_RAW;

-- Total Number of Null values for all columns
SELECT COUNT(*) FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES
WHERE SUPPLIERCATEGORYID IS NULL
OR SUPPLIERCATEGORYNAME IS NULL;
    
-- Null value checks for all columns (only numeric/boolean data type columns)
SELECT 
    COUNT(CASE WHEN SUPPLIERCATEGORYID IS NULL THEN 1 END) AS count_SUPPLIERCATEGORYID_NULL,
    COUNT(CASE WHEN SUPPLIERCATEGORYNAME IS NULL THEN 1 END) AS count_SUPPLIERCATEGORYNAME_NULL
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES;


-- data type conversion (to number) SUPPLIERCATEGORYID 

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES
ADD COLUMN SUPPLIERCATEGORYID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES
SET SUPPLIERCATEGORYID_NUM = TO_NUMBER(SUPPLIERCATEGORYID);

SELECT SUPPLIERCATEGORYID, SUPPLIERCATEGORYID_NUM
FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES
DROP COLUMN SUPPLIERCATEGORYID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES
RENAME COLUMN SUPPLIERCATEGORYID_NUM TO SUPPLIERCATEGORYID;


-- primary key
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES
ADD CONSTRAINT PK_PURCHASING_SUPPLIERCATEGORIES_SUPPLIERCATEGORYID
PRIMARY KEY (SUPPLIERCATEGORYID);

-- unique key
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES
ADD CONSTRAINT UK_PURCHASING_SUPPLIERCATEGORIES_SUPPLIERCATEGORYNAME
UNIQUE (SUPPLIERCATEGORYNAME);


-- null error handling for primary key (increment 1 to impute null value primary keys from the previous record)
WITH CTE AS (
    SELECT
        SUPPLIERCATEGORYID,
        SUPPLIERCATEGORYNAME,
        LAG(SUPPLIERCATEGORYID) OVER (ORDER BY SUPPLIERCATEGORYID) AS prev_SUPPLIERCATEGORYID,
        ROW_NUMBER() OVER (ORDER BY SUPPLIERCATEGORYID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES
)
SELECT 
    SUPPLIERCATEGORYNAME,
    CASE
        WHEN SUPPLIERCATEGORYID IS NULL THEN prev_SUPPLIERCATEGORYID + 1
        ELSE SUPPLIERCATEGORYID
    END AS SUPPLIERCATEGORYID
FROM CTE
ORDER BY row_num;

-- foreign key
ALTER TABLE KN_LOGISTICS.SNOWSQL.PURCHASING_Suppliers
ADD CONSTRAINT FK_Purchasing_Suppliers_SupplierCategoryID_Purchasing_SupplierCategories
FOREIGN KEY (SupplierCategoryID)
REFERENCES KN_LOGISTICS.SNOWSQL.PURCHASING_SUPPLIERCATEGORIES(SupplierCategoryID);

In [ ]:
-- Create a clean table with the same structure as the raw table
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS AS
SELECT
  CAST(CASE WHEN CUSTOMERID = 'NULL' THEN NULL ELSE CUSTOMERID END AS VARCHAR(38)) AS CUSTOMERID,
  CAST(CASE WHEN CUSTOMERNAME = 'NULL' THEN NULL ELSE CUSTOMERNAME END AS VARCHAR(100)) AS CUSTOMERNAME,
  CAST(CASE WHEN BILLTOCUSTOMERID = 'NULL' THEN NULL ELSE BILLTOCUSTOMERID END AS VARCHAR(38)) AS BILLTOCUSTOMERID,
  CAST(CASE WHEN CUSTOMERCATEGORYID = 'NULL' THEN NULL ELSE CUSTOMERCATEGORYID END AS VARCHAR(38)) AS CUSTOMERCATEGORYID,
  CAST(CASE WHEN BUYINGGROUPID = 'NULL' THEN NULL ELSE BUYINGGROUPID END AS VARCHAR(38)) AS BUYINGGROUPID,
  CAST(CASE WHEN PRIMARYCONTACTPERSONID = 'NULL' THEN NULL ELSE PRIMARYCONTACTPERSONID END AS VARCHAR(38)) AS PRIMARYCONTACTPERSONID,
  CAST(CASE WHEN ALTERNATECONTACTPERSONID = 'NULL' THEN NULL ELSE ALTERNATECONTACTPERSONID END AS VARCHAR(38)) AS ALTERNATECONTACTPERSONID,
  CAST(CASE WHEN DELIVERYMETHODID = 'NULL' THEN NULL ELSE DELIVERYMETHODID END AS VARCHAR(38)) AS DELIVERYMETHODID,
  CAST(CASE WHEN DELIVERYCITYID = 'NULL' THEN NULL ELSE DELIVERYCITYID END AS VARCHAR(38)) AS DELIVERYCITYID,
  CAST(CASE WHEN CREDITLIMIT = 'NULL' THEN NULL ELSE CREDITLIMIT END AS VARCHAR(38)) AS CREDITLIMIT,
  CAST(CASE WHEN ACCOUNTOPENEDDATE = 'NULL' THEN NULL ELSE ACCOUNTOPENEDDATE END AS VARCHAR(20)) AS ACCOUNTOPENEDDATE,
  CAST(CASE WHEN STANDARDDISCOUNTPERCENTAGE = 'NULL' THEN NULL ELSE STANDARDDISCOUNTPERCENTAGE END AS VARCHAR(38)) AS STANDARDDISCOUNTPERCENTAGE,
  CAST(CASE WHEN ISSTATEMENTSENT = 'NULL' THEN NULL ELSE ISSTATEMENTSENT END AS VARCHAR(38)) AS ISSTATEMENTSENT,
  CAST(CASE WHEN ISONCREDITHOLD = 'NULL' THEN NULL ELSE ISONCREDITHOLD END AS VARCHAR(38)) AS ISONCREDITHOLD,
  CAST(CASE WHEN PAYMENTDAYS = 'NULL' THEN NULL ELSE PAYMENTDAYS END AS VARCHAR(38)) AS PAYMENTDAYS,
  CAST(CASE WHEN PHONENUMBER = 'NULL' THEN NULL ELSE PHONENUMBER END AS VARCHAR(20)) AS PHONENUMBER,
  CAST(CASE WHEN WEBSITEURL = 'NULL' THEN NULL ELSE WEBSITEURL END AS VARCHAR(256)) AS WEBSITEURL
FROM KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS_RAW;

SELECT *
FROM KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS;

----------------------------------------------
--Converting of datatypes
----------------------------------------------

---------------------CUSTOMERID------------------------
-- Convert CUSTOMERID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN CUSTOMERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET CUSTOMERID_NUM = CAST(CUSTOMERID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN CUSTOMERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN CUSTOMERID_NUM TO CUSTOMERID;

---------------------BILLTOCUSTOMERID------------------------
-- Convert BILLTOCUSTOMERID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN BILLTOCUSTOMERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET BILLTOCUSTOMERID_NUM = CAST(BILLTOCUSTOMERID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN BILLTOCUSTOMERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN BILLTOCUSTOMERID_NUM TO BILLTOCUSTOMERID;

---------------------CUSTOMERCATEGORYID------------------------
-- Convert CUSTOMERCATEGORYID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN CUSTOMERCATEGORYID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET CUSTOMERCATEGORYID_NUM = CAST(CUSTOMERCATEGORYID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN CUSTOMERCATEGORYID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN CUSTOMERCATEGORYID_NUM TO CUSTOMERCATEGORYID;

---------------------BUYINGGROUPID------------------------
-- Convert BUYINGGROUPID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN BUYINGGROUPID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET BUYINGGROUPID_NUM = CAST(BUYINGGROUPID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN BUYINGGROUPID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN BUYINGGROUPID_NUM TO BUYINGGROUPID;

---------------------PRIMARYCONTACTPERSONID------------------------
-- Convert PRIMARYCONTACTPERSONID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN PRIMARYCONTACTPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET PRIMARYCONTACTPERSONID_NUM = CAST(PRIMARYCONTACTPERSONID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN PRIMARYCONTACTPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN PRIMARYCONTACTPERSONID_NUM TO PRIMARYCONTACTPERSONID;

---------------------ALTERNATECONTACTPERSONID------------------------
-- Convert ALTERNATECONTACTPERSONID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN ALTERNATECONTACTPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET ALTERNATECONTACTPERSONID_NUM = CAST(ALTERNATECONTACTPERSONID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN ALTERNATECONTACTPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN ALTERNATECONTACTPERSONID_NUM TO ALTERNATECONTACTPERSONID;

---------------------DELIVERYMETHODID------------------------
-- Convert DELIVERYMETHODID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN DELIVERYMETHODID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET DELIVERYMETHODID_NUM = CAST(DELIVERYMETHODID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN DELIVERYMETHODID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN DELIVERYMETHODID_NUM TO DELIVERYMETHODID;

---------------------DELIVERYCITYID------------------------
-- Convert DELIVERYCITYID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN DELIVERYCITYID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET DELIVERYCITYID_NUM = CAST(DELIVERYCITYID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN DELIVERYCITYID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN DELIVERYCITYID_NUM TO DELIVERYCITYID;

---------------------CREDITLIMIT------------------------
-- Convert CREDITLIMIT to DECIMAL(18,2)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN CREDITLIMIT_DECIMAL DECIMAL(18,2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET CREDITLIMIT_DECIMAL = CAST(CREDITLIMIT AS DECIMAL(18,2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN CREDITLIMIT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN CREDITLIMIT_DECIMAL TO CREDITLIMIT;

---------------------ACCOUNTOPENDDATE------------------------
-- Convert ACCOUNTOPENDDATE to DATE
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN ACCOUNTOPENEDATE_DATE DATE;

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET ACCOUNTOPENEDATE_DATE = CAST(ACCOUNTOPENEDDATE AS DATE);

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN ACCOUNTOPENEDDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN ACCOUNTOPENEDATE_DATE TO ACCOUNTOPENEDDATE;

---------------------STANDARDDISCOUNTPERCENTAGE------------------------
-- Convert STANDARDDISCOUNTPERCENTAGE to DECIMAL(18,2)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN STANDARDDISCOUNTPERCENTAGE_DECIMAL DECIMAL(18,2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET STANDARDDISCOUNTPERCENTAGE_DECIMAL = CAST(STANDARDDISCOUNTPERCENTAGE AS DECIMAL(18,2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN STANDARDDISCOUNTPERCENTAGE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN STANDARDDISCOUNTPERCENTAGE_DECIMAL TO STANDARDDISCOUNTPERCENTAGE;

---------------------ISSTATEMENT------------------------
-- Convert ISSTATEMENTSENT to BOOLEAN
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN ISSTATEMENT_BOOL BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET ISSTATEMENT_BOOL = CASE 
    WHEN ISSTATEMENTSENT = '1' THEN TRUE
    ELSE FALSE
END;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN ISSTATEMENTSENT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN ISSTATEMENT_BOOL TO ISSTATEMENTSENT;

---------------------ISONCREDITHOLD------------------------
-- Convert ISONCREDITHOLD to BOOLEAN
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN ISONCREDITHOLD_BOOL BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET ISONCREDITHOLD_BOOL = CASE 
    WHEN ISONCREDITHOLD = '1' THEN TRUE
    ELSE FALSE
END;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN ISONCREDITHOLD;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN ISONCREDITHOLD_BOOL TO ISONCREDITHOLD;

---------------------PAYMENTDAYS------------------------
-- Convert PAYMENTDAYS to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD COLUMN PAYMENTDAYS_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
SET PAYMENTDAYS_NUM = CAST(PAYMENTDAYS AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
DROP COLUMN PAYMENTDAYS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
RENAME COLUMN PAYMENTDAYS_NUM TO PAYMENTDAYS;

SELECT 
    CUSTOMERID,
    BILLTOCUSTOMERID,
    CUSTOMERCATEGORYID,
    BUYINGGROUPID,
    PRIMARYCONTACTPERSONID,
    ALTERNATECONTACTPERSONID,
    DELIVERYMETHODID,
    DELIVERYCITYID,
    CREDITLIMIT,
    ACCOUNTOPENEDDATE,
    STANDARDDISCOUNTPERCENTAGE,
    ISSTATEMENTSENT,
    ISONCREDITHOLD,
    PAYMENTDAYS
FROM KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
LIMIT 10;


--------------------------------------------------
--Adding of primary key into the table
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD CONSTRAINT PK_SALES_CUSTOMERS_CUSTOMERID
PRIMARY KEY (CUSTOMERID);


--------------------------------------------------
--Adding of unique key into the table
--------------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD CONSTRAINT UK_SALES_CUSTOMERS_CUSTOMERNAME
UNIQUE (CUSTOMERNAME);


-------------------------------------------------------
--ERROR HANDLING
-------------------------------------------------------
WITH CTE AS (
    SELECT 
        CUSTOMERID,
        CUSTOMERNAME, 
        BILLTOCUSTOMERID, 
        CUSTOMERCATEGORYID, 
        BUYINGGROUPID, 
        PRIMARYCONTACTPERSONID, 
        ALTERNATECONTACTPERSONID, 
        DELIVERYMETHODID, 
        DELIVERYCITYID, 
        CREDITLIMIT, 
        ACCOUNTOPENEDDATE, 
        STANDARDDISCOUNTPERCENTAGE, 
        ISSTATEMENTSENT, 
        ISONCREDITHOLD, 
        PAYMENTDAYS, 
        PHONENUMBER, 
        WEBSITEURL,
        LAG(CUSTOMERID) OVER (ORDER BY CUSTOMERID) AS prev_customerid,
        ROW_NUMBER() OVER (ORDER BY CUSTOMERID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
)
SELECT  
        CUSTOMERNAME, 
        BILLTOCUSTOMERID, 
        CUSTOMERCATEGORYID, 
        BUYINGGROUPID, 
        PRIMARYCONTACTPERSONID, 
        ALTERNATECONTACTPERSONID, 
        DELIVERYMETHODID, 
        DELIVERYCITYID, 
        CREDITLIMIT, 
        ACCOUNTOPENEDDATE, 
        STANDARDDISCOUNTPERCENTAGE, 
        ISSTATEMENTSENT, 
        ISONCREDITHOLD, 
        PAYMENTDAYS, 
        PHONENUMBER, 
        WEBSITEURL,
    CASE
        WHEN CUSTOMERID IS NULL THEN prev_customerid + 1
        ELSE CUSTOMERID
    END AS CUSTOMERID
FROM CTE
ORDER BY row_num;

-------------------------------------------------------
--Adding of foreign key to table
-------------------------------------------------------

-- Foreign Key: CustomerCategoryID -> Sales.CustomerCategories.CustomerCategoryID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD CONSTRAINT FK_SALES_CUSTOMERS_CUSTOMERCATEGORYID_SALES_CUSTOMERCATEGORIES
FOREIGN KEY (CUSTOMERCATEGORYID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERCATEGORIES(CUSTOMERCATEGORYID);

-- Foreign Key: BuyingGroupID -> Sales.BuyingGroups.BuyingGroupID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD CONSTRAINT FK_SALES_CUSTOMERS_BUYINGGROUPID_SALES_BUYINGGROUPS
FOREIGN KEY (BUYINGGROUPID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_BUYINGGROUPS(BUYINGGROUPID);

-- Foreign Key: PrimaryContactPersonID -> Application.People.PersonID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD CONSTRAINT FK_SALES_CUSTOMERS_PRIMARYCONTACTPERSONID_APPLICATION_PEOPLE
FOREIGN KEY (PRIMARYCONTACTPERSONID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PERSONID);

-- Foreign Key: AlternateContactPersonID -> Application.People.PersonID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD CONSTRAINT FK_SALES_CUSTOMERS_ALTERNATECONTACTPERSONID_APPLICATION_PEOPLE
FOREIGN KEY (ALTERNATECONTACTPERSONID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PERSONID);

-- Foreign Key: DeliveryMethodID -> Application.DeliveryMethods.DeliveryMethodID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS
ADD CONSTRAINT FK_SALES_CUSTOMERS_DELIVERYMETHODID_APPLICATION_DELIVERYMETHODS
FOREIGN KEY (DELIVERYMETHODID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_DELIVERYMETHODS(DELIVERYMETHODID);

SELECT * FROM SALES_CUSTOMERS LIMIT 20

#### Cleaning SALES_ORDERLINES

In [ ]:
-- Create a clean table with the same structure as the raw table
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES AS
SELECT
  CAST(CASE WHEN ORDERLINEID = 'NULL' THEN NULL ELSE ORDERLINEID END AS VARCHAR(38)) AS ORDERLINEID,
  CAST(CASE WHEN ORDERID = 'NULL' THEN NULL ELSE ORDERID END AS VARCHAR(38)) AS ORDERID,
  CAST(CASE WHEN STOCKITEMID = 'NULL' THEN NULL ELSE STOCKITEMID END AS VARCHAR(38)) AS STOCKITEMID,
  CAST(CASE WHEN DESCRIPTION = 'NULL' THEN NULL ELSE DESCRIPTION END AS VARCHAR(100)) AS DESCRIPTION,
  CAST(CASE WHEN PACKAGETYPEID = 'NULL' THEN NULL ELSE PACKAGETYPEID END AS VARCHAR(38)) AS PACKAGETYPEID,
  CAST(CASE WHEN QUANTITY = 'NULL' THEN NULL ELSE QUANTITY END AS VARCHAR(38)) AS QUANTITY,
  CAST(CASE WHEN UNITPRICE = 'NULL' THEN NULL ELSE UNITPRICE END AS VARCHAR(38)) AS UNITPRICE,
  CAST(CASE WHEN TAXRATE = 'NULL' THEN NULL ELSE TAXRATE END AS VARCHAR(38)) AS TAXRATE,
  CAST(CASE WHEN PICKEDQUANTITY = 'NULL' THEN NULL ELSE PICKEDQUANTITY END AS VARCHAR(38)) AS PICKEDQUANTITY,
  CAST(CASE WHEN PICKINGCOMPLETEDWHEN = 'NULL' THEN NULL ELSE PICKINGCOMPLETEDWHEN END AS VARCHAR(25)) AS PICKINGCOMPLETEDWHEN
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES_RAW;


------------------------------------
--Converting of datatypes
------------------------------------
---------------------ORDERLINEID------------------------
-- Convert ORDERLINEID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN ORDERLINEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET ORDERLINEID_NUM = CAST(ORDERLINEID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN ORDERLINEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN ORDERLINEID_NUM TO ORDERLINEID;

---------------------ORDERID------------------------
-- Convert ORDERID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN ORDERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET ORDERID_NUM = CAST(ORDERID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN ORDERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN ORDERID_NUM TO ORDERID;

---------------------STOCKITEMID------------------------
-- Convert STOCKITEMID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN STOCKITEMID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET STOCKITEMID_NUM = CAST(STOCKITEMID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN STOCKITEMID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN STOCKITEMID_NUM TO STOCKITEMID;

---------------------PACKAGETYPEID------------------------
-- Convert PACKAGETYPEID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN PACKAGETYPEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET PACKAGETYPEID_NUM = CAST(PACKAGETYPEID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN PACKAGETYPEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN PACKAGETYPEID_NUM TO PACKAGETYPEID;

---------------------QUANTITY------------------------
-- Convert QUANTITY to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN QUANTITY_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET QUANTITY_NUM = CAST(QUANTITY AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN QUANTITY;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN QUANTITY_NUM TO QUANTITY;

---------------------PICKEDQUANTITY------------------------
-- Convert PICKEDQUANTITY to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN PICKEDQUANTITY_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET PICKEDQUANTITY_NUM = CAST(PICKEDQUANTITY AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN PICKEDQUANTITY;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN PICKEDQUANTITY_NUM TO PICKEDQUANTITY;

---------------------UNITPRICE------------------------
-- Add a new column for UNITPRICE with DECIMAL(18,2)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN UNITPRICE_DECIMAL DECIMAL(18,2);

-- Populate the new column with converted values
UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET UNITPRICE_DECIMAL = CAST(UNITPRICE AS DECIMAL(18,2));

-- Verify the conversion
SELECT UNITPRICE, UNITPRICE_DECIMAL
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
LIMIT 10;

-- Drop the old UNITPRICE column
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN UNITPRICE;

-- Rename the new column to the original column name
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN UNITPRICE_DECIMAL TO UNITPRICE;

---------------------TAXRATE------------------------
-- Add a new column for TAXRATE with DECIMAL(18,3)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN TAXRATE_DECIMAL DECIMAL(18,3);

-- Populate the new column with converted values
UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET TAXRATE_DECIMAL = CAST(TAXRATE AS DECIMAL(18,3));

-- Verify the conversion
SELECT TAXRATE, TAXRATE_DECIMAL
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
LIMIT 10;

-- Drop the old TAXRATE column
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN TAXRATE;

-- Rename the new column to the original column name
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN TAXRATE_DECIMAL TO TAXRATE;


------------PICKINGCOMPLETEDWHEN----------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD COLUMN PICKINGCOMPLETEDWHEN_DATE TIMESTAMP;

// Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
SET PICKINGCOMPLETEDWHEN_DATE = TO_TIMESTAMP(PICKINGCOMPLETEDWHEN, 'DD/MM/YYYY HH24:MI');

SELECT PICKINGCOMPLETEDWHEN, PICKINGCOMPLETEDWHEN_DATE
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
DROP COLUMN PICKINGCOMPLETEDWHEN;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
RENAME COLUMN PICKINGCOMPLETEDWHEN_DATE TO PICKINGCOMPLETEDWHEN;

// Check 10 rows of TRANSACTIONOCCURREDWHEN after updating the data type
SELECT PICKINGCOMPLETEDWHEN FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES LIMIT 10;

// Check 10 rows of the whole table after updating the data type
SELECT * FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES LIMIT 10;



-------------------------------------
--Adding primary key to table
-------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD CONSTRAINT PK_SALES_ORDERLINES_ORDERLINEID
PRIMARY KEY (ORDERLINEID);

-----------------------------------------------------
--ERROR HANDLING
-----------------------------------------------------
WITH CTE AS (
    SELECT 
        ORDERLINEID,
        ORDERID,
        STOCKITEMID,
        DESCRIPTION,
        PACKAGETYPEID,
        QUANTITY,
        UNITPRICE,
        TAXRATE,
        PICKEDQUANTITY,
        PICKINGCOMPLETEDWHEN,
        LAG(ORDERLINEID) OVER (ORDER BY ORDERLINEID) AS prev_orderlineid,
        ROW_NUMBER() OVER (ORDER BY ORDERLINEID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
)
SELECT 
    ORDERID,
    STOCKITEMID,
    DESCRIPTION,
    PACKAGETYPEID,
    QUANTITY,
    UNITPRICE,
    TAXRATE,
    PICKEDQUANTITY,
    PICKINGCOMPLETEDWHEN,
    CASE
        WHEN ORDERLINEID IS NULL THEN prev_orderlineid + 1
        ELSE ORDERLINEID
    END AS ORDERLINEID
FROM CTE
ORDER BY row_num;



-------------------------------------------------------
--Adding of foreign key to table
-------------------------------------------------------
-- Add foreign key for OrderID referencing Sales.Orders
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD CONSTRAINT FK_OrderID FOREIGN KEY (OrderID)
REFERENCES SALES_ORDERS(OrderID);

-- Add foreign key for StockItemID referencing Warehouse.StockItems
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD CONSTRAINT FK_StockItemID FOREIGN KEY (StockItemID)
REFERENCES Warehouse_StockItems(StockItemID);

-- Add foreign key for PackageTypeID referencing Warehouse.PackageTypes
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD CONSTRAINT FK_PackageTypeID FOREIGN KEY (PackageTypeID)
REFERENCES Warehouse_PackageTypes(PackageTypeID);

---------------------FINAL VERIFICATION------------------------
-- Select all columns to verify their final state
SELECT *
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
LIMIT 10;

#### Cleaning SALES_ORDERS

In [ ]:
-- Create a clean table with the same structure as the raw table but we pull data from the raw table into this table (final table)
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS AS
SELECT
  CAST(CASE WHEN ORDERID = 'NULL' THEN NULL ELSE ORDERID END AS VARCHAR(38)) AS ORDERID,
  CAST(CASE WHEN CUSTOMERID = 'NULL' THEN NULL ELSE CUSTOMERID END AS VARCHAR(38)) AS CUSTOMERID,
  CAST(CASE WHEN SALESPERSONPERSONID = 'NULL' THEN NULL ELSE SALESPERSONPERSONID END AS VARCHAR(38)) AS SALESPERSONPERSONID,
  CAST(CASE WHEN PICKEDBYPERSONID = 'NULL' THEN NULL ELSE PICKEDBYPERSONID END AS VARCHAR(38)) AS PICKEDBYPERSONID,
  CAST(CASE WHEN CONTACTPERSONID = 'NULL' THEN NULL ELSE CONTACTPERSONID END AS VARCHAR(38)) AS CONTACTPERSONID,
  CAST(CASE WHEN BACKORDERORDERID = 'NULL' THEN NULL ELSE BACKORDERORDERID END AS VARCHAR(38)) AS BACKORDERORDERID,
  CAST(CASE WHEN ORDERDATE = 'NULL' THEN NULL ELSE ORDERDATE END AS VARCHAR(20)) AS ORDERDATE,
  CAST(CASE WHEN EXPECTEDDELIVERYDATE = 'NULL' THEN NULL ELSE EXPECTEDDELIVERYDATE END AS VARCHAR(20)) AS EXPECTEDDELIVERYDATE,
  CAST(CASE WHEN CUSTOMERPURCHASEORDERNUMBER = 'NULL' THEN NULL ELSE CUSTOMERPURCHASEORDERNUMBER END AS VARCHAR(20)) AS CUSTOMERPURCHASEORDERNUMBER,
  CAST(CASE WHEN ISUNDERSUPPLYBACKORDERED = 'NULL' THEN NULL ELSE ISUNDERSUPPLYBACKORDERED END AS VARCHAR(38)) AS ISUNDERSUPPLYBACKORDERED,
  CAST(CASE WHEN PICKINGCOMPLETEDWHEN = 'NULL' THEN NULL ELSE PICKINGCOMPLETEDWHEN END AS VARCHAR(25)) AS PICKINGCOMPLETEDWHEN
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS_RAW;

--Drop column BACKORDERORDERID (unneccesary column)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN BACKORDERORDERID;

-- Null value checks for all columns
SELECT 
    COUNT(CASE WHEN ISUNDERSUPPLYBACKORDERED IS NULL THEN 1 END) AS count_ISUNDERSUPPLYBACKORDERED_NULL,
    COUNT(CASE WHEN ORDERID IS NULL THEN 1 END) AS count_ORDERID_NULL,    
    COUNT(CASE WHEN CUSTOMERID IS NULL THEN 1 END) AS count_CUSTOMERID_NULL,
    COUNT(CASE WHEN SALESPERSONPERSONID IS NULL THEN 1 END) AS count_SALESPERSONPERSONID_NULL,
    COUNT(CASE WHEN PICKEDBYPERSONID IS NULL THEN 1 END) AS count_PICKEDBYPERSONID_NULL,
    COUNT(CASE WHEN CONTACTPERSONID IS NULL THEN 1 END) AS count_CONTACTPERSONID_NULL,    
    COUNT(CASE WHEN ORDERDATE IS NULL THEN 1 END) AS count_ORDERDATE_NULL,
    COUNT(CASE WHEN EXPECTEDDELIVERYDATE IS NULL THEN 1 END) AS count_EXPECTEDDELIVERYDATE_NULL,    
    COUNT(CASE WHEN CUSTOMERPURCHASEORDERNUMBER IS NULL THEN 1 END) AS count_CUSTOMERPURCHASEORDERNUMBER_NULL,
    COUNT(CASE WHEN PICKINGCOMPLETEDWHEN IS NULL THEN 1 END) AS count_PICKINGCOMPLETEDWHEN_NULL
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

---------------------------------------
--ORDERDATE
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN ORDERDATE_DATE DATE;

-- Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET ORDERDATE_DATE = TO_DATE(ORDERDATE, 'DD/MM/YYYY');

SELECT ORDERDATE, ORDERDATE_DATE
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN ORDERDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN ORDERDATE_DATE TO ORDERDATE;


---------------------------------------
-- EXPECTEDDELIVERYDATE
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN EXPECTEDDELIVERYDATE_DATE DATE;

-- Change to date format for EXPECTEDDELIVERYDATE column
UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET EXPECTEDDELIVERYDATE_DATE = TO_DATE(EXPECTEDDELIVERYDATE, 'DD/MM/YYYY');

SELECT EXPECTEDDELIVERYDATE, EXPECTEDDELIVERYDATE_DATE
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN EXPECTEDDELIVERYDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN EXPECTEDDELIVERYDATE_DATE TO EXPECTEDDELIVERYDATE;

---------------------------------------
-- PICKINGCOMPLETEDWHEN
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN PICKINGCOMPLETEDWHEN_DATE TIMESTAMP;

-- Change to date format for PICKINGCOMPLETEDWHEN column
UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET PICKINGCOMPLETEDWHEN_DATE = TO_TIMESTAMP(PICKINGCOMPLETEDWHEN, 'DD/MM/YYYY HH24:MI');

SELECT PICKINGCOMPLETEDWHEN, PICKINGCOMPLETEDWHEN_DATE
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN PICKINGCOMPLETEDWHEN;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN PICKINGCOMPLETEDWHEN_DATE TO PICKINGCOMPLETEDWHEN;

---------------------------------------
--PICKEDBYPERSONID
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN PICKEDBYPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET PICKEDBYPERSONID_NUM = TO_NUMBER(PICKEDBYPERSONID);

SELECT PICKEDBYPERSONID, PICKEDBYPERSONID_NUM
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN PICKEDBYPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN PICKEDBYPERSONID_NUM TO PICKEDBYPERSONID;

--check the number of rows of PICKEDBYPERSONID = 18(random PICKEDBYPERSONID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS
WHERE PICKEDBYPERSONID = 18;

---------------------------------------
-- CONTACTPERSONID
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN CONTACTPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET CONTACTPERSONID_NUM = TO_NUMBER(CONTACTPERSONID);

SELECT CONTACTPERSONID, CONTACTPERSONID_NUM
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN CONTACTPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN CONTACTPERSONID_NUM TO CONTACTPERSONID;

--check the number of rows of CONTACTPERSONID = 3178(random CONTACTPERSONID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS
WHERE CONTACTPERSONID = 3178;

---------------------------------------
-- SALESPERSONPERSONID
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN SALESPERSONPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET SALESPERSONPERSONID_NUM = TO_NUMBER(SALESPERSONPERSONID);

SELECT SALESPERSONPERSONID, SALESPERSONPERSONID_NUM
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN SALESPERSONPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN SALESPERSONPERSONID_NUM TO SALESPERSONPERSONID;

--check the number of rows of SALESPERSONPERSONID = 2(random CONTACTPERSONID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS
WHERE SALESPERSONPERSONID = 2;


---------------------------------------
--CUSTOMERID
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN CUSTOMERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET CUSTOMERID_NUM = TO_NUMBER(CUSTOMERID);

SELECT CUSTOMERID, CUSTOMERID_NUM
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN CUSTOMERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN CUSTOMERID_NUM TO CUSTOMERID;

--check the number of rows of customerid = 832(random customerid from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS
WHERE CUSTOMERID = 832;

---------------------------------------
--ORDERID
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN ORDERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET ORDERID_NUM = TO_NUMBER(ORDERID);

SELECT ORDERID, ORDERID_NUM
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN ORDERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN ORDERID_NUM TO ORDERID;

--check the number of rows of ORDERID = 109(random ORDERID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS
WHERE ORDERID = 109;

---------------------------------------
-- ISUNDERSUPPLYBACKORDERED 
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD COLUMN ISUNDERSUPPLYBACKORDERED_NUM BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
SET ISUNDERSUPPLYBACKORDERED_NUM = TO_BOOLEAN(ISUNDERSUPPLYBACKORDERED);

SELECT ISUNDERSUPPLYBACKORDERED, ISUNDERSUPPLYBACKORDERED_NUM
FROM KN_LOGISTICS.SNOWSQL.SALES_ORDERS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
DROP COLUMN ISUNDERSUPPLYBACKORDERED;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
RENAME COLUMN ISUNDERSUPPLYBACKORDERED_NUM TO ISUNDERSUPPLYBACKORDERED;


--primary key
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD CONSTRAINT PK_SALES_ORDERS_ORDERID
PRIMARY KEY (ORDERID);

-- null error handling for primary key (increment 1 to impute null value primary keys from the previous record)
WITH CTE AS (
    SELECT 
        OrderID,
        CustomerID,
        SalespersonPersonID,
        PickedByPersonID,
        ContactPersonID,
        OrderDate,
        ExpectedDeliveryDate,
        CustomerPurchaseOrderNumber,
        IsUndersupplyBackordered,
        PickingCompletedWhen,
        LAG(OrderID) OVER (ORDER BY OrderID) AS prev_orderid,
        ROW_NUMBER() OVER (ORDER BY OrderID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.Sales_Orders
)
SELECT 
    CustomerID,
    SalespersonPersonID,
    PickedByPersonID,
    ContactPersonID,
    OrderDate,
    ExpectedDeliveryDate,
    CustomerPurchaseOrderNumber,
    IsUndersupplyBackordered,
    PickingCompletedWhen,
    CASE
        WHEN OrderID IS NULL THEN prev_orderid + 1
        ELSE OrderID
    END AS OrderID
FROM CTE
ORDER BY row_num;

-- foreign key
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD CONSTRAINT FK_Sales_Orders_CustomerID_Sales_Customers
FOREIGN KEY (CustomerID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS(CustomerID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD CONSTRAINT FK_Sales_Orders_ContactPersonID_Application_People
FOREIGN KEY (ContactPersonID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PersonID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD CONSTRAINT FK_Sales_Orders_PickedByPersonID_Application_People
FOREIGN KEY (PickedByPersonID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PersonID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERS
ADD CONSTRAINT FK_Sales_Orders_SalespersonPersonID_Application_People
FOREIGN KEY (SalespersonPersonID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PersonID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_OrderID_Sales_Orders
FOREIGN KEY (OrderID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_ORDERS(OrderID);

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_ORDERLINES
ADD CONSTRAINT FK_Sales_OrderLines_OrderID_Sales_Orders
FOREIGN KEY (OrderID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_ORDERS(OrderID);

#### Cleaning WAREHOUSE_COLDROOMTEMPERATURES

In [ ]:
-- Create a clean table with the same structure as the raw table
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES AS
SELECT
  CAST(CASE WHEN COLDROOMTEMPERATUREID = 'NULL' THEN NULL ELSE COLDROOMTEMPERATUREID END AS VARCHAR(38)) AS COLDROOMTEMPERATUREID,
  CAST(CASE WHEN COLDROOMSENSORNUMBER = 'NULL' THEN NULL ELSE COLDROOMSENSORNUMBER END AS VARCHAR(38)) AS COLDROOMSENSORNUMBER,
  CAST(CASE WHEN RECORDEDWHEN = 'NULL' THEN NULL ELSE RECORDEDWHEN END AS VARCHAR(25)) AS RECORDEDWHEN,
  CAST(CASE WHEN TEMPERATURE = 'NULL' THEN NULL ELSE TEMPERATURE END AS VARCHAR(38)) AS TEMPERATURE
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES_RAW;

-- Verify the clean table
SELECT *
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES;

-----------------------------------------------
-- Conversion of data types in table
-----------------------------------------------

--------------------------------------------------
// data type conversion (to timestamp)
--------------------------------------------------
// TRANSACTIONOCCURREDWHEN
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
ADD COLUMN RECORDEDWHEN_T TIMESTAMP;

// Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
SET RECORDEDWHEN_T = TO_TIMESTAMP(RECORDEDWHEN, 'DD/MM/YYYY HH24:MI');

SELECT RECORDEDWHEN, RECORDEDWHEN_T
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
DROP COLUMN RECORDEDWHEN;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
RENAME COLUMN RECORDEDWHEN_T TO RECORDEDWHEN;

// Check 10 rows of the whole table after updating the data type
SELECT * FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES;


//COLDROOMTEMPERATUREID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
ADD COLUMN COLDROOMTEMPERATUREID_NUMBER NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
SET COLDROOMTEMPERATUREID_NUMBER = TO_NUMBER(COLDROOMTEMPERATUREID);

SELECT COLDROOMTEMPERATUREID, COLDROOMTEMPERATUREID_NUMBER
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
DROP COLUMN COLDROOMTEMPERATUREID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
RENAME COLUMN COLDROOMTEMPERATUREID_NUMBER TO COLDROOMTEMPERATUREID;

//COLDROOMTEMPERATURES
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
ADD COLUMN COLDROOMSENSORNUMBER_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
SET COLDROOMSENSORNUMBER_NUM = TO_NUMBER(COLDROOMSENSORNUMBER);

SELECT COLDROOMSENSORNUMBER_NUM, COLDROOMSENSORNUMBER
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
DROP COLUMN COLDROOMSENSORNUMBER;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
RENAME COLUMN COLDROOMSENSORNUMBER_NUM TO COLDROOMSENSORNUMBER;

//TEMPERATURE 
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
ADD COLUMN TEMPERATURE_FLOAT FLOAT;

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
SET TEMPERATURE_FLOAT = CAST(TEMPERATURE AS FLOAT);

SELECT TEMPERATURE_FLOAT, TEMPERATURE
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
DROP COLUMN TEMPERATURE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
RENAME COLUMN TEMPERATURE_FLOAT TO TEMPERATURE;


-- Select the transformed data for verification
SELECT 
    COLDROOMTEMPERATUREID,
    COLDROOMSENSORNUMBER,
    RECORDEDWHEN,
    TEMPERATURE
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
LIMIT 10;

--------------------------------------------
-- Adding of primary key to table
--------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
ADD CONSTRAINT PK_WAREHOUSE_COLDROOMTEMPERATURES_COLDROOMTEMPERATUREID
PRIMARY KEY (COLDROOMTEMPERATUREID);

-------------------------------------------------------
-- ERROR HANDLING
-------------------------------------------------------
WITH CTE AS (
    SELECT 
        COLDROOMTEMPERATUREID,
        COLDROOMSENSORNUMBER,
        RECORDEDWHEN,
        TEMPERATURE,
        LAG(COLDROOMTEMPERATUREID) OVER (ORDER BY COLDROOMTEMPERATUREID) AS prev_coldroomtemperatureid,
        ROW_NUMBER() OVER (ORDER BY COLDROOMTEMPERATUREID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_COLDROOMTEMPERATURES
)
SELECT 
    COLDROOMSENSORNUMBER,
    RECORDEDWHEN,
    TEMPERATURE,
    CASE
        WHEN COLDROOMTEMPERATUREID IS NULL THEN prev_coldroomtemperatureid + 1
        ELSE COLDROOMTEMPERATUREID
    END AS COLDROOMTEMPERATUREID
FROM CTE
ORDER BY row_num;


-------------------------------------------------------
--Adding of foreign key to table
-------------------------------------------------------
-- THERE ARE NO FOREIGN KEYS IN THIS TABLE


#### Cleaning WAREHOUSE_STOCKITEMHOLDINGS

In [ ]:
-- Create a clean table with the same structure as the raw table but we pull data from the raw table into this table (final table)
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS AS
SELECT
  CAST(CASE WHEN STOCKITEMID = 'NULL' THEN NULL ELSE STOCKITEMID END AS VARCHAR(38)) AS STOCKITEMID,
  CAST(CASE WHEN QUANTITYONHAND = 'NULL' THEN NULL ELSE QUANTITYONHAND END AS VARCHAR(38)) AS QUANTITYONHAND,
  CAST(CASE WHEN BINLOCATION = 'NULL' THEN NULL ELSE BINLOCATION END AS VARCHAR(20)) AS BINLOCATION,
  CAST(CASE WHEN LASTSTOCKTAKEQUANTITY = 'NULL' THEN NULL ELSE LASTSTOCKTAKEQUANTITY END AS VARCHAR(38)) AS LASTSTOCKTAKEQUANTITY,
  CAST(CASE WHEN LASTCOSTPRICE = 'NULL' THEN NULL ELSE LASTCOSTPRICE END AS VARCHAR(38)) AS LASTCOSTPRICE,
  CAST(CASE WHEN REORDERLEVEL = 'NULL' THEN NULL ELSE REORDERLEVEL END AS VARCHAR(38)) AS REORDERLEVEL,
  CAST(CASE WHEN TARGETSTOCKLEVEL = 'NULL' THEN NULL ELSE TARGETSTOCKLEVEL END AS VARCHAR(38)) AS TARGETSTOCKLEVEL
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS_RAW;

-- Total Number of Null values for all columns
SELECT COUNT(*) FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
WHERE STOCKITEMID IS NULL
OR QUANTITYONHAND IS NULL
OR BINLOCATION IS NULL
OR LASTSTOCKTAKEQUANTITY IS NULL
OR LASTCOSTPRICE IS NULL
OR REORDERLEVEL IS NULL
OR TARGETSTOCKLEVEL IS NULL;
    
-- Null value checks for all columns (only numeric data type columns)
SELECT 
    COUNT(CASE WHEN STOCKITEMID IS NULL THEN 1 END) AS count_STOCKITEMID_NULL,
    COUNT(CASE WHEN QUANTITYONHAND IS NULL THEN 1 END) AS count_STOCKITEMID_NULL,
    COUNT(CASE WHEN LASTSTOCKTAKEQUANTITY IS NULL THEN 1 END) AS count_LASTSTOCKTAKEQUANTITY_NULL,
    COUNT(CASE WHEN BINLOCATION IS NULL THEN 1 END) AS count_BINLOCATION_NULL,    
    COUNT(CASE WHEN LASTCOSTPRICE IS NULL THEN 1 END) AS count_LASTCOSTPRICE_NULL,
    COUNT(CASE WHEN REORDERLEVEL IS NULL THEN 1 END) AS count_REORDERLEVEL_NULL,
    COUNT(CASE WHEN TARGETSTOCKLEVEL IS NULL THEN 1 END) AS count_TARGETSTOCKLEVEL_NULL
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS;

------------------------------------------------------
-- STOCKITEMID
------------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
ADD COLUMN STOCKITEMID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
SET STOCKITEMID_NUM = TO_NUMBER(STOCKITEMID);

SELECT STOCKITEMID, STOCKITEMID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
DROP COLUMN STOCKITEMID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
RENAME COLUMN STOCKITEMID_NUM TO STOCKITEMID;

--------------------------------------------------------
-- QUANTITYONHAND
------------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
ADD COLUMN QUANTITYONHAND_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
SET QUANTITYONHAND_NUM = TO_NUMBER(QUANTITYONHAND);

SELECT QUANTITYONHAND, QUANTITYONHAND_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
DROP COLUMN QUANTITYONHAND;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
RENAME COLUMN QUANTITYONHAND_NUM TO QUANTITYONHAND;


-----------------------------------------
-- LASTSTOCKTAKEQUANTITY
------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
ADD COLUMN LASTSTOCKTAKEQUANTITY_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
SET LASTSTOCKTAKEQUANTITY_NUM = TO_NUMBER(LASTSTOCKTAKEQUANTITY);

SELECT LASTSTOCKTAKEQUANTITY, LASTSTOCKTAKEQUANTITY_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
DROP COLUMN LASTSTOCKTAKEQUANTITY;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
RENAME COLUMN LASTSTOCKTAKEQUANTITY_NUM TO LASTSTOCKTAKEQUANTITY;


-----------------------------------------
-- LastCostPrice
--------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
ADD COLUMN LASTCOSTPRICE_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
SET LASTCOSTPRICE_NUM = TO_DECIMAL(LASTCOSTPRICE, 18, 2);

SELECT LASTCOSTPRICE, LASTCOSTPRICE_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
DROP COLUMN LASTCOSTPRICE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
RENAME COLUMN LASTCOSTPRICE_NUM TO LASTCOSTPRICE;



--------------------------------------
-- REORDERLEVEL
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
ADD COLUMN REORDERLEVEL_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
SET REORDERLEVEL_NUM = TO_NUMBER(REORDERLEVEL);

SELECT REORDERLEVEL, REORDERLEVEL_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
DROP COLUMN REORDERLEVEL;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
RENAME COLUMN REORDERLEVEL_NUM TO REORDERLEVEL;


-------------------------------------
-- TARGETSTOCKLEVEL
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
ADD COLUMN TARGETSTOCKLEVEL_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
SET TARGETSTOCKLEVEL_NUM = TO_NUMBER(TARGETSTOCKLEVEL);

SELECT TARGETSTOCKLEVEL, TARGETSTOCKLEVEL_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
DROP COLUMN TARGETSTOCKLEVEL;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
RENAME COLUMN TARGETSTOCKLEVEL_NUM TO TARGETSTOCKLEVEL;


-- primary key
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMHOLDINGS
ADD CONSTRAINT PK_WAREHOUSE_STOCKITEMHOLDINGS_STOCKITEMID
PRIMARY KEY (STOCKITEMID);


-- null error handling for primary key (increment 1 to impute null value primary keys from the previous record)
WITH CTE AS (
    SELECT 
        StockItemID,
        QuantityOnHand,
        BinLocation,
        LastStocktakeQuantity,
        LastCostPrice,
        ReorderLevel,
        TargetStockLevel,
        LAG(StockItemID) OVER (ORDER BY StockItemID) AS prev_stockitemid,
        ROW_NUMBER() OVER (ORDER BY StockItemID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.Warehouse_StockItemHoldings
)
SELECT 
    QuantityOnHand,
    BinLocation,
    LastStocktakeQuantity,
    LastCostPrice,
    ReorderLevel,
    TargetStockLevel,
    CASE
        WHEN StockItemID IS NULL THEN prev_stockitemid + 1
        ELSE StockItemID
    END AS StockItemID
FROM CTE
ORDER BY row_num;
-- foreign keys
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItemHoldings
ADD CONSTRAINT FK_Warehouse_StockItemHoldings_StockItemID_Warehouse_StockItems
FOREIGN KEY (StockItemID)
REFERENCES KN_LOGISTICS.SNOWSQL.Warehouse_StockItems(StockItemID);

#### Cleaning WAREHOUSE_STOCKITEMS

In [ ]:
-- Create a clean table with the same structure as the raw table but we pull data from the raw table into this table (final table)
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS AS
SELECT
  CAST(CASE WHEN STOCKITEMID = 'NULL' THEN NULL ELSE STOCKITEMID END AS VARCHAR(38)) AS STOCKITEMID,
  CAST(CASE WHEN STOCKITEMNAME = 'NULL' THEN NULL ELSE STOCKITEMNAME END AS VARCHAR(100)) AS STOCKITEMNAME,
  CAST(CASE WHEN SUPPLIERID = 'NULL' THEN NULL ELSE SUPPLIERID END AS VARCHAR(38)) AS SUPPLIERID,
  CAST(CASE WHEN COLORID = 'NULL' THEN NULL ELSE COLORID END AS VARCHAR(38)) AS COLORID,
  CAST(CASE WHEN UNITPACKAGEID = 'NULL' THEN NULL ELSE UNITPACKAGEID END AS VARCHAR(38)) AS UNITPACKAGEID,
  CAST(CASE WHEN OUTERPACKAGEID = 'NULL' THEN NULL ELSE OUTERPACKAGEID END AS VARCHAR(38)) AS OUTERPACKAGEID,
  CAST(CASE WHEN BRAND = 'NULL' THEN NULL ELSE BRAND END AS VARCHAR(50)) AS BRAND,
  CAST(CASE WHEN SIZE = 'NULL' THEN NULL ELSE SIZE END AS VARCHAR(20)) AS SIZE,
  CAST(CASE WHEN LEADTIMEDAYS = 'NULL' THEN NULL ELSE LEADTIMEDAYS END AS VARCHAR(38)) AS LEADTIMEDAYS,
  CAST(CASE WHEN QUANTITYPEROUTER = 'NULL' THEN NULL ELSE QUANTITYPEROUTER END AS VARCHAR(38)) AS QUANTITYPEROUTER,
  CAST(CASE WHEN ISCHILLERSTOCK = 'NULL' THEN NULL ELSE ISCHILLERSTOCK END AS VARCHAR(38)) AS ISCHILLERSTOCK,
  CAST(CASE WHEN BARCODE = 'NULL' THEN NULL ELSE BARCODE END AS VARCHAR(50)) AS BARCODE,
  CAST(CASE WHEN TAXRATE = 'NULL' THEN NULL ELSE TAXRATE END AS VARCHAR(38)) AS TAXRATE,
  CAST(CASE WHEN UNITPRICE = 'NULL' THEN NULL ELSE UNITPRICE END AS VARCHAR(38)) AS UNITPRICE,
  CAST(CASE WHEN RECOMMENDEDRETAILPRICE = 'NULL' THEN NULL ELSE RECOMMENDEDRETAILPRICE END AS VARCHAR(38)) AS RECOMMENDEDRETAILPRICE,
  CAST(CASE WHEN TYPICALWEIGHTPERUNIT = 'NULL' THEN NULL ELSE TYPICALWEIGHTPERUNIT END AS VARCHAR(38)) AS TYPICALWEIGHTPERUNIT
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS_RAW;


--Drop column barcode (unneccesary column)
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN BARCODE;
    

-- Null value checks for all columns (only numeric/boolean data type columns)
SELECT 
    SUM(CASE WHEN STOCKITEMID IS NULL THEN 1 END) AS count_STOCKITEMID_NULL,
    SUM(CASE WHEN SUPPLIERID IS NULL THEN 1 END) AS count_SUPPLIERID_NULL,
    SUM(CASE WHEN UNITPACKAGEID IS NULL THEN 1 END) AS count_UNITPACKAGEID_NULL,
    SUM(CASE WHEN STOCKITEMNAME IS NULL THEN 1 END) AS count_STOCKITEMNAME_NULL,
    SUM(CASE WHEN BRAND IS NULL THEN 1 END) AS count_BRAND_NULL,
    SUM(CASE WHEN SIZE IS NULL THEN 1 END) AS count_SIZE_NULL,    
    SUM(CASE WHEN OUTERPACKAGEID IS NULL THEN 1 END) AS count_OUTERPACKAGEID_NULL,
    SUM(CASE WHEN LEADTIMEDAYS IS NULL THEN 1 END) AS count_LEADTIMEDAYS_NULL,
    SUM(CASE WHEN QUANTITYPEROUTER IS NULL THEN 1 END) AS count_QUANTITYPEROUTER_NULL,
    SUM(CASE WHEN ISCHILLERSTOCK IS NULL THEN 1 END) AS count_ISCHILLERSTOCK_NULL,
    SUM(CASE WHEN TAXRATE IS NULL THEN 1 END) AS count_TAXRATE_NULL,
    SUM(CASE WHEN UNITPRICE IS NULL THEN 1 END) AS count_UNITPRICE_NULL,
    SUM(CASE WHEN RECOMMENDEDRETAILPRICE IS NULL THEN 1 END) AS count_RECOMMENDEDRETAILPRICE_NULL,
    SUM(CASE WHEN TYPICALWEIGHTPERUNIT IS NULL THEN 1 END) AS count_TYPICALWEIGHTPERUNIT_NULL
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;


-- Update Size column
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET SIZE = 'No Size'
WHERE SIZE IS NULL;

-- Update Brand column
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET BRAND = 'No Brand'
WHERE BRAND IS NULL;

-- Assign COLORID = 40 for items with specific text, but avoid overwriting existing correct COLORIDs
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 40 -- No Color
WHERE COLORID IS NULL -- Only update items with no assigned COLORID
  AND (
      STOCKITEMNAME ILIKE '%food flash drive%' 
      OR STOCKITEMNAME ILIKE '%Pack of 12 action figures%' 
      OR STOCKITEMNAME ILIKE '%bubblewrap%' 
      OR STOCKITEMNAME ILIKE '%bubble wrap%'
      OR STOCKITEMNAME ILIKE '%replacement blades%'
      OR STOCKITEMNAME ILIKE '%Air cushion film%'
      OR STOCKITEMNAME ILIKE '%Novelty chilli chocolates%'
      OR STOCKITEMNAME ILIKE '%Chocolate%'
      OR STOCKITEMNAME ILIKE '%White chocolate%'
  );

-- Assign Transparent ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 41 -- Transparent
WHERE STOCKITEMNAME ILIKE '%Clear packaging tape%';

-- Assign Multicolor ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 42 -- Multicolor
WHERE STOCKITEMNAME ILIKE '%Black and Orange%'
   OR STOCKITEMNAME ILIKE '%Black and yellow%'
   OR STOCKITEMNAME ILIKE '%Red and white%';
   
-- Assign ColorIDs based on keywords with brackets (e.g., (Azure), (Beige), (Black), etc.)
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET ColorID = 
    CASE 
        WHEN LOWER(StockItemName) LIKE '%(azure)%' THEN 1
        WHEN LOWER(StockItemName) LIKE '%(beige)%' THEN 2
        WHEN LOWER(StockItemName) LIKE '%(black)%' THEN 3
        WHEN LOWER(StockItemName) LIKE '%(blue)%' THEN 4
        WHEN LOWER(StockItemName) LIKE '%(charcoal)%' THEN 5
        WHEN LOWER(StockItemName) LIKE '%(chartreuse)%' THEN 6
        WHEN LOWER(StockItemName) LIKE '%(cyan)%' THEN 7
        WHEN LOWER(StockItemName) LIKE '%(dark brown)%' THEN 8
        WHEN LOWER(StockItemName) LIKE '%(dark green)%' THEN 9
        WHEN LOWER(StockItemName) LIKE '%(fuchsia)%' THEN 10
        WHEN LOWER(StockItemName) LIKE '%(gold)%' THEN 11
        WHEN LOWER(StockItemName) LIKE '%(steel gray)%' THEN 12
        WHEN LOWER(StockItemName) LIKE '%(hot pink)%' THEN 13
        WHEN LOWER(StockItemName) LIKE '%(indigo)%' THEN 14
        WHEN LOWER(StockItemName) LIKE '%(ivory)%' THEN 15
        WHEN LOWER(StockItemName) LIKE '%(khaki)%' THEN 16
        WHEN LOWER(StockItemName) LIKE '%(lavender)%' THEN 17
        WHEN LOWER(StockItemName) LIKE '%(light brown)%' THEN 18
        WHEN LOWER(StockItemName) LIKE '%(light green)%' THEN 19
        WHEN LOWER(StockItemName) LIKE '%(maroon)%' THEN 20
        WHEN LOWER(StockItemName) LIKE '%(mauve)%' THEN 21
        WHEN LOWER(StockItemName) LIKE '%(navy blue)%' THEN 22
        WHEN LOWER(StockItemName) LIKE '%(olive)%' THEN 23
        WHEN LOWER(StockItemName) LIKE '%(orange)%' THEN 24
        WHEN LOWER(StockItemName) LIKE '%(plum)%' THEN 25
        WHEN LOWER(StockItemName) LIKE '%(puce)%' THEN 26
        WHEN LOWER(StockItemName) LIKE '%(purple)%' THEN 27
        WHEN LOWER(StockItemName) LIKE '%(red)%' THEN 28
        WHEN LOWER(StockItemName) LIKE '%(royal blue)%' THEN 29
        WHEN LOWER(StockItemName) LIKE '%(salmon)%' THEN 30
        WHEN LOWER(StockItemName) LIKE '%(silver)%' THEN 31
        WHEN LOWER(StockItemName) LIKE '%(tan)%' THEN 32
        WHEN LOWER(StockItemName) LIKE '%(teal)%' THEN 33
        WHEN LOWER(StockItemName) LIKE '%(wheat)%' THEN 34
        WHEN LOWER(StockItemName) LIKE '%(white)%' THEN 35
        WHEN LOWER(StockItemName) LIKE '%(yellow)%' THEN 36
        ELSE ColorID -- Retain the existing ColorID if no match
    END
WHERE ColorID IS NULL;

-- Assign Black ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 3 -- Black
WHERE STOCKITEMNAME ILIKE '%Permanent marker black%';

-- Assign Red ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 28 -- Red
WHERE STOCKITEMNAME ILIKE '%Permanent marker red%';

-- Assign Blue ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 4 -- Blue
WHERE STOCKITEMNAME ILIKE '%Permanent marker blue%';

-- Assign Generic Green ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 37 -- Green
WHERE STOCKITEMNAME ILIKE '%Green%'
  AND COLORID IS NULL; -- Ensure only unassigned items are updated

-- Assign Pink ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 38 -- Pink
WHERE STOCKITEMNAME ILIKE '%Pink%'
  AND COLORID IS NULL; -- Ensure only unassigned items are updated

-- Assign Brown ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 39 -- Brown
WHERE STOCKITEMNAME ILIKE '%Brown%'
  AND COLORID IS NULL;

-- Assign Blue ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 4 -- Blue
WHERE STOCKITEMNAME ILIKE '%(Blue)%'
  AND COLORID IS NULL;

-- Assign Blue ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 4 -- Blue
WHERE STOCKITEMNAME ILIKE '%Blue%'
  AND COLORID IS NULL;

-- Assign Blue ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 43 -- Gray
WHERE STOCKITEMNAME ILIKE '%Gray%'
  AND COLORID IS NULL;


-- Assign Generic Green ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 28 -- Red
WHERE STOCKITEMNAME ILIKE '%Red%'
  AND COLORID IS NULL; -- Ensure only unassigned items are updated


-- Assign Generic Green ColorID
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 3 -- Black
WHERE STOCKITEMNAME ILIKE '%Black%'
  AND COLORID IS NULL; -- Ensure only unassigned items are updated

-- Handle remaining NULL values in Color
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID = 40 -- No Color
WHERE COLORID IS NULL;

-- Null value checks for all columns (only varchar data type columns)
SELECT
    COUNT(CASE WHEN STOCKITEMNAME  IS NULL THEN 1 END) AS NULL_COUNT_STOCKITEMNAME,    
    COUNT(CASE WHEN COLORID IS NULL THEN 1 END) AS NULL_COUNT_COLORID,
    COUNT(CASE WHEN BRAND IS NULL THEN 1 END) AS NULL_COUNT_BRAND,
    COUNT(CASE WHEN SIZE  IS NULL THEN 1 END) AS NULL_COUNT_SIZE
FROM 
    KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;
    

-- Null value checks for all columns (only numeric/boolean data type columns)
SELECT 
    COUNT(CASE WHEN STOCKITEMID IS NULL THEN 1 END) AS count_STOCKITEMID_NULL,
    COUNT(CASE WHEN SUPPLIERID IS NULL THEN 1 END) AS count_SUPPLIERID_NULL,
    COUNT(CASE WHEN UNITPACKAGEID IS NULL THEN 1 END) AS count_UNITPACKAGEID_NULL,
    COUNT(CASE WHEN OUTERPACKAGEID IS NULL THEN 1 END) AS count_OUTERPACKAGEID_NULL,
    COUNT(CASE WHEN LEADTIMEDAYS IS NULL THEN 1 END) AS count_LEADTIMEDAYS_NULL,
    COUNT(CASE WHEN QUANTITYPEROUTER IS NULL THEN 1 END) AS count_QUANTITYPEROUTER_NULL,
    COUNT(CASE WHEN ISCHILLERSTOCK IS NULL THEN 1 END) AS count_ISCHILLERSTOCK_NULL,
    COUNT(CASE WHEN TAXRATE IS NULL THEN 1 END) AS count_TAXRATE_NULL,
    COUNT(CASE WHEN UNITPRICE IS NULL THEN 1 END) AS count_UNITPRICE_NULL,
    COUNT(CASE WHEN RECOMMENDEDRETAILPRICE IS NULL THEN 1 END) AS count_RECOMMENDEDRETAILPRICE_NULL,
    COUNT(CASE WHEN TYPICALWEIGHTPERUNIT IS NULL THEN 1 END) AS count_TYPICALWEIGHTPERUNIT_NULL
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;


-- STOCKITEMID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN STOCKITEMID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET STOCKITEMID_NUM = TO_NUMBER(STOCKITEMID);

SELECT STOCKITEMID, STOCKITEMID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN STOCKITEMID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN STOCKITEMID_NUM TO STOCKITEMID;


-- SUPPLIERID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN SUPPLIERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET SUPPLIERID_NUM = TO_NUMBER(SUPPLIERID);

SELECT SUPPLIERID, SUPPLIERID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN SUPPLIERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN SUPPLIERID_NUM TO SUPPLIERID;

--check the number of rows of SUPPLIERID = 7(random SUPPLIERID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
WHERE SUPPLIERID = 7;



-- COLORID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN COLORID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET COLORID_NUM = TO_NUMBER(COLORID);

SELECT COLORID, COLORID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN COLORID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN COLORID_NUM TO COLORID;

--check the number of rows of SUPPLIERID = 7(random SUPPLIERID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
WHERE COLORID = 28;



-- UNITPACKAGEID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN UNITPACKAGEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET UNITPACKAGEID_NUM = TO_NUMBER(UNITPACKAGEID);

SELECT UNITPACKAGEID, UNITPACKAGEID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN UNITPACKAGEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN UNITPACKAGEID_NUM TO UNITPACKAGEID;




-- OUTERPACKAGEID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN OUTERPACKAGEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET OUTERPACKAGEID_NUM = TO_NUMBER(OUTERPACKAGEID);

SELECT OUTERPACKAGEID, OUTERPACKAGEID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN OUTERPACKAGEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN OUTERPACKAGEID_NUM TO OUTERPACKAGEID;




-- LEADTIMEDAYS
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN LEADTIMEDAYS_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET LEADTIMEDAYS_NUM = TO_NUMBER(LEADTIMEDAYS);

SELECT LEADTIMEDAYS, LEADTIMEDAYS_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN LEADTIMEDAYS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN LEADTIMEDAYS_NUM TO LEADTIMEDAYS;




-- QUANTITYPEROUTER
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN QUANTITYPEROUTER_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET QUANTITYPEROUTER_NUM = TO_NUMBER(QUANTITYPEROUTER);

SELECT QUANTITYPEROUTER, QUANTITYPEROUTER_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN QUANTITYPEROUTER;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN QUANTITYPEROUTER_NUM TO QUANTITYPEROUTER;


-- TAXRATE
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN TAXRATE_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET TAXRATE_NUM = TO_DECIMAL(TAXRATE, 18, 2);

SELECT TAXRATE, TAXRATE_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN TAXRATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN TAXRATE_NUM TO TAXRATE;


-- UNITPRICE
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN UNITPRICE_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET UNITPRICE_NUM = TO_DECIMAL(UNITPRICE, 18, 2);

SELECT UNITPRICE, UNITPRICE_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN UNITPRICE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN UNITPRICE_NUM TO UNITPRICE;


--RECOMMENDEDRETAILPRICE
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN RECOMMENDEDRETAILPRICE_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET RECOMMENDEDRETAILPRICE_NUM = TO_DECIMAL(RECOMMENDEDRETAILPRICE, 18, 2);

SELECT RECOMMENDEDRETAILPRICE, RECOMMENDEDRETAILPRICE_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN RECOMMENDEDRETAILPRICE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN RECOMMENDEDRETAILPRICE_NUM TO RECOMMENDEDRETAILPRICE;


--TYPICALWEIGHTPERUNIT
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN TYPICALWEIGHTPERUNIT_NUM DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET TYPICALWEIGHTPERUNIT_NUM = TO_DECIMAL(TYPICALWEIGHTPERUNIT, 18, 2);

SELECT TYPICALWEIGHTPERUNIT, TYPICALWEIGHTPERUNIT_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN TYPICALWEIGHTPERUNIT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN TYPICALWEIGHTPERUNIT_NUM TO TYPICALWEIGHTPERUNIT;


-- TYPICALWEIGHTPERUNIT 
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD COLUMN IsChillerStock_NUM BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
SET IsChillerStock_NUM = TO_BOOLEAN(IsChillerStock);

SELECT IsChillerStock, IsChillerStock_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
DROP COLUMN IsChillerStock;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
RENAME COLUMN IsChillerStock_NUM TO IsChillerStock;



-- primary key
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD CONSTRAINT PK_WAREHOUSE_STOCKITEMS_STOCKITEMID
PRIMARY KEY (STOCKITEMID);

--unique key
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD CONSTRAINT UK_WAREHOUSE_STOCKITEMS_STOCKITEMNAME
UNIQUE (STOCKITEMNAME);


-- null error handling for primary key (increment 1 to impute null value primary keys from the previous record)
WITH CTE AS (
    SELECT 
        StockItemID,
        StockItemName,
        SupplierID,
        ColorID,
        UnitPackageID,
        OuterPackageID,
        Brand,
        Size,
        LeadTimeDays,
        QuantityPerOuter,
        IsChillerStock,
        TaxRate,
        UnitPrice,
        RecommendedRetailPrice,
        TypicalWeightPerUnit,
        LAG(StockItemID) OVER (ORDER BY StockItemID) AS prev_stockitemid,
        ROW_NUMBER() OVER (ORDER BY StockItemID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.Warehouse_StockItems
)
SELECT 
    StockItemName,
    SupplierID,
    ColorID,
    UnitPackageID,
    OuterPackageID,
    Brand,
    Size,
    LeadTimeDays,
    QuantityPerOuter,
    IsChillerStock,
    TaxRate,
    UnitPrice,
    RecommendedRetailPrice,
    TypicalWeightPerUnit,
    CASE
        WHEN StockItemID IS NULL THEN prev_stockitemid + 1
        ELSE StockItemID
    END AS StockItemID
FROM CTE
ORDER BY row_num;

-- Foreign Key: Warehouse.StockItems.SupplierID -> Purchasing.Suppliers.SupplierID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS
ADD CONSTRAINT FK_Warehouse_StockItems_SupplierID_Purchasing_Suppliers
FOREIGN KEY (SupplierID)
REFERENCES KN_LOGISTICS.SNOWSQL.Purchasing_Suppliers(SupplierID);

-- Foreign Key: Sales.InvoiceLines.StockItemID -> Warehouse.StockItems.StockItemID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Sales_InvoiceLines
ADD CONSTRAINT FK_Sales_InvoiceLines_StockItemID_Warehouse_StockItems
FOREIGN KEY (StockItemID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS(StockItemID);

-- Foreign Key: Sales.OrderLines.StockItemID -> Warehouse.StockItems.StockItemID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Sales_OrderLines
ADD CONSTRAINT FK_Sales_OrderLines_StockItemID_Warehouse_StockItems
FOREIGN KEY (StockItemID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS(StockItemID);

-- Foreign Key: Purchasing.PurchaseOrderLines.StockItemID -> Warehouse.StockItems.StockItemID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Purchasing_PurchaseOrderLines
ADD CONSTRAINT FK_Purchasing_PurchaseOrderLines_StockItemID_Warehouse_StockItems
FOREIGN KEY (StockItemID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS(StockItemID);

-- Foreign Key: Warehouse.StockItemHoldings.StockItemID -> Warehouse.StockItems.StockItemID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_StockItemHoldings
ADD CONSTRAINT PKFK_Warehouse_StockItemHoldings_StockItemID_Warehouse_StockItems
FOREIGN KEY (StockItemID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS(StockItemID);

-- Foreign Key: Warehouse.StockItemTransactions.StockItemID -> Warehouse.StockItems.StockItemID
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_StockItemTransactions
ADD CONSTRAINT FK_Warehouse_StockItemTransactions_StockItemID_Warehouse_StockItems
FOREIGN KEY (StockItemID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS(StockItemID);

-- Foreign key for ColorID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItems
ADD CONSTRAINT FK_Warehouse_StockItems_ColorID_Warehouse_Colors
FOREIGN KEY (ColorID)
REFERENCES KN_LOGISTICS.SNOWSQL.Warehouse_Colors (ColorID);

-- Foreign key for OuterPackageID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItems
ADD CONSTRAINT FK_Warehouse_StockItems_OuterPackageID_Warehouse_PackageTypes
FOREIGN KEY (OuterPackageID)
REFERENCES KN_LOGISTICS.SNOWSQL.Warehouse_PackageTypes (PackageTypeID);

-- Foreign key for UnitPackageID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItems
ADD CONSTRAINT FK_Warehouse_StockItems_UnitPackageID_Warehouse_PackageTypes
FOREIGN KEY (UnitPackageID)
REFERENCES KN_LOGISTICS.SNOWSQL.Warehouse_PackageTypes (PackageTypeID);

#### Clean WAREHOUSE_VEHICLETEMPERATURES

In [ ]:
-- Create a clean table with the same structure as the raw table
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES AS
SELECT
  CAST(CASE WHEN VEHICLETEMPERATUREID = 'NULL' THEN NULL ELSE VEHICLETEMPERATUREID END AS VARCHAR(38)) AS VEHICLETEMPERATUREID,
  CAST(CASE WHEN VEHICLEREGISTRATION = 'NULL' THEN NULL ELSE VEHICLEREGISTRATION END AS VARCHAR(255)) AS VEHICLEREGISTRATION,
  CAST(CASE WHEN CHILLERSENSORNUMBER = 'NULL' THEN NULL ELSE CHILLERSENSORNUMBER END AS VARCHAR(38)) AS CHILLERSENSORNUMBER,
  CAST(CASE WHEN RECORDEDWHEN = 'NULL' THEN NULL ELSE RECORDEDWHEN END AS VARCHAR(25)) AS RECORDEDWHEN,
  CAST(CASE WHEN TEMPERATURE = 'NULL' THEN NULL ELSE TEMPERATURE END AS VARCHAR(38)) AS TEMPERATURE,
  CAST(CASE WHEN FULLSENSORDATA = 'NULL' THEN NULL ELSE FULLSENSORDATA END AS VARCHAR(1000)) AS FULLSENSORDATA
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES_RAW;

-- Verify the clean table
SELECT *
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES;

------------------------------------
--Conversion of datatype
------------------------------------
---------------VEHICLETEMPERATUREID----------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
ADD COLUMN VEHICLETEMPERATUREID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
SET VEHICLETEMPERATUREID_NUM = CAST(VEHICLETEMPERATUREID AS NUMBER(38,0));

SELECT VEHICLETEMPERATUREID_NUM, VEHICLETEMPERATUREID
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
LIMIT 10;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
DROP COLUMN VEHICLETEMPERATUREID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
RENAME COLUMN VEHICLETEMPERATUREID_NUM TO VEHICLETEMPERATUREID;


---------------------CHILLERSENSORNUMBER-----------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
ADD COLUMN CHILLERSENSORNUMBER_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
SET CHILLERSENSORNUMBER_NUM = CAST(CHILLERSENSORNUMBER AS NUMBER(38,0));

SELECT CHILLERSENSORNUMBER_NUM, CHILLERSENSORNUMBER
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
LIMIT 10;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
DROP COLUMN CHILLERSENSORNUMBER;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
RENAME COLUMN CHILLERSENSORNUMBER_NUM TO CHILLERSENSORNUMBER;


--------------------TEMPERATURE-----------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
ADD COLUMN TEMPERATURE_DECIMAL DECIMAL(10,2);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
SET TEMPERATURE_DECIMAL = CAST(TEMPERATURE AS DECIMAL(10,2));

SELECT TEMPERATURE_DECIMAL, TEMPERATURE
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
LIMIT 10;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
DROP COLUMN TEMPERATURE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
RENAME COLUMN TEMPERATURE_DECIMAL TO TEMPERATURE;


---------------------RECORDEDWHEN-----------------------------------
--------------------------------------------------
// data type conversion (to timestamp)
--------------------------------------------------
// TRANSACTIONOCCURREDWHEN
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
ADD COLUMN RECORDEDWHEN_DATE TIMESTAMP;

// Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
SET RECORDEDWHEN_DATE = TO_TIMESTAMP(RECORDEDWHEN,'DD/MM/YYYY HH24:MI');

SELECT RECORDEDWHEN, RECORDEDWHEN_DATE
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
DROP COLUMN RECORDEDWHEN;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
RENAME COLUMN RECORDEDWHEN_DATE TO RECORDEDWHEN;

// Check 10 rows of TRANSACTIONOCCURREDWHEN after updating the data type
SELECT RECORDEDWHEN FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES LIMIT 10;

// Check 10 rows of the whole table after updating the data type
SELECT * FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES LIMIT 10;




// Check 10 rows of the whole table after updating the data type
SELECT * FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES;
SELECT 
    VEHICLETEMPERATUREID,
    CHILLERSENSORNUMBER,
    TEMPERATURE,
    RECORDEDWHEN
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
LIMIT 20;


-------------------------------------------
--Addition of primary key to table
-------------------------------------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
ADD CONSTRAINT PK_WAREHOUSE_VEHICLETEMPERATURES_VEHICLETEMPERATUREID
PRIMARY KEY (VEHICLETEMPERATUREID);

-------------------------------------------------------
-- ERROR HANDLING 
-------------------------------------------------------
WITH CTE AS (
    SELECT 
        VEHICLETEMPERATUREID,
        VEHICLEREGISTRATION,
        CHILLERSENSORNUMBER,
        RECORDEDWHEN,
        TEMPERATURE,
        FULLSENSORDATA,
        LAG(VEHICLETEMPERATUREID) OVER (ORDER BY VEHICLETEMPERATUREID) AS prev_vehicletemperatureid,
        ROW_NUMBER() OVER (ORDER BY VEHICLETEMPERATUREID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_VEHICLETEMPERATURES
)
SELECT 
        VEHICLEREGISTRATION,
        CHILLERSENSORNUMBER,
        RECORDEDWHEN,
        TEMPERATURE,
        FULLSENSORDATA,
    CASE
        WHEN VEHICLETEMPERATUREID IS NULL THEN prev_vehicletemperatureid + 1
        ELSE VEHICLETEMPERATUREID
    END AS VEHICLETEMPERATUREID

FROM CTE
ORDER BY row_num;

-------------------------------------------------------
--Adding of foreign key to table
-------------------------------------------------------
--  THERE ARE NO FOREIGN KEYS IN THIS TABLE

#### Cleaning SALES_INVOICES

In [ ]:
-- Create a clean table with the same structure as the raw table
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES AS
SELECT
  CAST(CASE WHEN INVOICEID = 'NULL' THEN NULL ELSE INVOICEID END AS VARCHAR(38)) AS INVOICEID,
  CAST(CASE WHEN CUSTOMERID = 'NULL' THEN NULL ELSE CUSTOMERID END AS VARCHAR(38)) AS CUSTOMERID,
  CAST(CASE WHEN BILLTOCUSTOMERID = 'NULL' THEN NULL ELSE BILLTOCUSTOMERID END AS VARCHAR(38)) AS BILLTOCUSTOMERID,
  CAST(CASE WHEN ORDERID = 'NULL' THEN NULL ELSE ORDERID END AS VARCHAR(38)) AS ORDERID,
  CAST(CASE WHEN DELIVERYMETHODID = 'NULL' THEN NULL ELSE DELIVERYMETHODID END AS VARCHAR(38)) AS DELIVERYMETHODID,
  CAST(CASE WHEN CONTACTPERSONID = 'NULL' THEN NULL ELSE CONTACTPERSONID END AS VARCHAR(38)) AS CONTACTPERSONID,
  CAST(CASE WHEN ACCOUNTSPERSONID = 'NULL' THEN NULL ELSE ACCOUNTSPERSONID END AS VARCHAR(38)) AS ACCOUNTSPERSONID,
  CAST(CASE WHEN SALESPERSONPERSONID = 'NULL' THEN NULL ELSE SALESPERSONPERSONID END AS VARCHAR(38)) AS SALESPERSONPERSONID,
  CAST(CASE WHEN PACKEDBYPERSONID = 'NULL' THEN NULL ELSE PACKEDBYPERSONID END AS VARCHAR(38)) AS PACKEDBYPERSONID,
  CAST(CASE WHEN INVOICEDATE = 'NULL' THEN NULL ELSE INVOICEDATE END AS VARCHAR(20)) AS INVOICEDATE,
  CAST(CASE WHEN CUSTOMERPURCHASEORDERNUMBER = 'NULL' THEN NULL ELSE CUSTOMERPURCHASEORDERNUMBER END AS VARCHAR(20)) AS CUSTOMERPURCHASEORDERNUMBER,
  CAST(CASE WHEN TOTALDRYITEMS = 'NULL' THEN NULL ELSE TOTALDRYITEMS END AS VARCHAR(38)) AS TOTALDRYITEMS,
  CAST(CASE WHEN TOTALCHILLERITEMS = 'NULL' THEN NULL ELSE TOTALCHILLERITEMS END AS VARCHAR(38)) AS TOTALCHILLERITEMS,
  CAST(CASE WHEN CONFIRMEDDELIVERYTIME = 'NULL' THEN NULL ELSE CONFIRMEDDELIVERYTIME END AS VARCHAR(25)) AS CONFIRMEDDELIVERYTIME,
  CAST(CASE WHEN CONFIRMEDRECEIVEDBY = 'NULL' THEN NULL ELSE CONFIRMEDRECEIVEDBY END AS VARCHAR(4000)) AS CONFIRMEDRECEIVEDBY
FROM KN_LOGISTICS.SNOWSQL.SALES_INVOICES_RAW;

SELECT *
FROM KN_LOGISTICS.SNOWSQL.SALES_INVOICES;


------------------------------------
--Converting of datatypes
------------------------------------
---------------------INVOICEID------------------------
-- Convert INVOICEID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN INVOICEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET INVOICEID_NUM = CAST(INVOICEID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN INVOICEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN INVOICEID_NUM TO INVOICEID;

---------------------CUSTOMERID------------------------
-- Convert CUSTOMERID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN CUSTOMERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET CUSTOMERID_NUM = CAST(CUSTOMERID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN CUSTOMERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN CUSTOMERID_NUM TO CUSTOMERID;

---------------------BILLTOCUSTOMERID------------------------
-- Convert BILLTOCUSTOMERID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN BILLTOCUSTOMERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET BILLTOCUSTOMERID_NUM = CAST(BILLTOCUSTOMERID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN BILLTOCUSTOMERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN BILLTOCUSTOMERID_NUM TO BILLTOCUSTOMERID;

---------------------ORDERID------------------------
-- Convert ORDERID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN ORDERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET ORDERID_NUM = CAST(ORDERID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN ORDERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN ORDERID_NUM TO ORDERID;

---------------------DELIVERYMETHODID------------------------
-- Convert DELIVERYMETHODID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN DELIVERYMETHODID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET DELIVERYMETHODID_NUM = CAST(DELIVERYMETHODID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN DELIVERYMETHODID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN DELIVERYMETHODID_NUM TO DELIVERYMETHODID;

---------------------CONTACTPERSONID------------------------
-- Convert CONTACTPERSONID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN CONTACTPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET CONTACTPERSONID_NUM = CAST(CONTACTPERSONID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN CONTACTPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN CONTACTPERSONID_NUM TO CONTACTPERSONID;

---------------------ACCOUNTSPERSONID------------------------
-- Convert ACCOUNTSPERSONID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN ACCOUNTSPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET ACCOUNTSPERSONID_NUM = CAST(ACCOUNTSPERSONID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN ACCOUNTSPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN ACCOUNTSPERSONID_NUM TO ACCOUNTSPERSONID;

---------------------SALESPERSONPERSONID------------------------
-- Convert SALESPERSONPERSONID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN SALESPERSONPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET SALESPERSONPERSONID_NUM = CAST(SALESPERSONPERSONID AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN SALESPERSONPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN SALESPERSONPERSONID_NUM TO SALESPERSONPERSONID;

---------------------PACKEDBYPERSONID------------------------
-- Convert PACKEDBYPERSONID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN PACKEDBYPERSONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET PACKEDBYPERSONID_NUM = CAST(PACKEDBYPERSONID AS NUMBER(38,0));


ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN PACKEDBYPERSONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN PACKEDBYPERSONID_NUM TO PACKEDBYPERSONID;

---------------------TOTALDRYITEMS------------------------
-- Convert TOTALDRYITEMS to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN TOTALDRYITEMS_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET TOTALDRYITEMS_NUM = CAST(TOTALDRYITEMS AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN TOTALDRYITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN TOTALDRYITEMS_NUM TO TOTALDRYITEMS;

---------------------TOTALCHILLERITEMS------------------------
-- Convert TOTALCHILLERITEMS to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN TOTALCHILLERITEMS_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET TOTALCHILLERITEMS_NUM = CAST(TOTALCHILLERITEMS AS NUMBER(38,0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN TOTALCHILLERITEMS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN TOTALCHILLERITEMS_NUM TO TOTALCHILLERITEMS;

---------------------INVOICEDATE------------------------
-- Add a new column with DATE data type
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN INVOICEDATE_DATE DATE;

-- Populate the new column with converted values using TO_DATE with the correct format
UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET INVOICEDATE_DATE = TO_DATE(INVOICEDATE, 'YYYY-MM-DD');


-- Drop the old INVOICEDATE column
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN INVOICEDATE;

-- Rename the new column to the original column name
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN INVOICEDATE_DATE TO INVOICEDATE;


------------------CONFIRMEDDELIVERYTIME----------------

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD COLUMN CONFIRMEDDELIVERYTIME_DATE TIMESTAMP;

// Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
SET CONFIRMEDDELIVERYTIME_DATE = TO_TIMESTAMP(CONFIRMEDDELIVERYTIME, 'YYYY-MM-DD HH24:MI:SS');

SELECT CONFIRMEDDELIVERYTIME, CONFIRMEDDELIVERYTIME_DATE
FROM KN_LOGISTICS.SNOWSQL.SALES_INVOICES;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
DROP COLUMN CONFIRMEDDELIVERYTIME;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
RENAME COLUMN CONFIRMEDDELIVERYTIME_DATE TO CONFIRMEDDELIVERYTIME;

// Check 10 rows of TRANSACTIONOCCURREDWHEN after updating the data type
SELECT CONFIRMEDDELIVERYTIME FROM KN_LOGISTICS.SNOWSQL.SALES_INVOICES LIMIT 10;

// Check 10 rows of the whole table after updating the data type
SELECT * FROM KN_LOGISTICS.SNOWSQL.SALES_INVOICES LIMIT 10;

-------------------------------------
--Adding primary key to table
-------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT PK_SALES_INVOICES_INVOICEID
PRIMARY KEY (INVOICEID);

------------------------------------------------------
--ERROR HANDLING
------------------------------------------------------
WITH CTE AS (
    SELECT 
        INVOICEID,
        CUSTOMERID, 
        BILLTOCUSTOMERID, 
        ORDERID, 
        DELIVERYMETHODID, 
        CONTACTPERSONID, 
        ACCOUNTSPERSONID, 
        SALESPERSONPERSONID, 
        PACKEDBYPERSONID, 
        INVOICEDATE, 
        CUSTOMERPURCHASEORDERNUMBER, 
        TOTALDRYITEMS, 
        TOTALCHILLERITEMS, 
        CONFIRMEDDELIVERYTIME, 
        CONFIRMEDRECEIVEDBY,
        LAG(INVOICEID) OVER (ORDER BY INVOICEID) AS prev_invoiceid,
        ROW_NUMBER() OVER (ORDER BY INVOICEID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.SALES_INVOICES
)
SELECT  
        CUSTOMERID, 
        BILLTOCUSTOMERID, 
        ORDERID, 
        DELIVERYMETHODID, 
        CONTACTPERSONID, 
        ACCOUNTSPERSONID, 
        SALESPERSONPERSONID, 
        PACKEDBYPERSONID, 
        INVOICEDATE, 
        CUSTOMERPURCHASEORDERNUMBER, 
        TOTALDRYITEMS, 
        TOTALCHILLERITEMS, 
        CONFIRMEDDELIVERYTIME, 
        CONFIRMEDRECEIVEDBY,
    CASE
        WHEN INVOICEID IS NULL THEN prev_invoiceid + 1
        ELSE INVOICEID
    END AS INVOICEID
FROM CTE
ORDER BY row_num;


-------------------------------------------------------
--Adding of foreign key to table
-------------------------------------------------------

-- Foreign Key: Sales.Invoices.CustomerID -> Sales.Customers.CustomerID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_CustomerID_Sales_Customers
FOREIGN KEY (CustomerID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS(CustomerID);

-- Foreign Key: Sales.Invoices.BillToCustomerID -> Sales.Customers.CustomerID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_BillToCustomerID_Sales_Customers
FOREIGN KEY (BillToCustomerID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS(CustomerID);

-- Foreign Key: Sales.Invoices.OrderID -> Sales.Orders.OrderID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_OrderID_Sales_Orders
FOREIGN KEY (OrderID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_ORDERS(OrderID);

-- Foreign Key: Sales.Invoices.DeliveryMethodID -> Application.DeliveryMethods.DeliveryMethodID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_DeliveryMethodID_Application_DeliveryMethods
FOREIGN KEY (DeliveryMethodID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_DELIVERYMETHODS(DeliveryMethodID);

-- Foreign Key: Sales.Invoices.ContactPersonID -> Application.People.PersonID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_ContactPersonID_Application_People
FOREIGN KEY (ContactPersonID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PersonID);

-- Foreign Key: Sales.Invoices.AccountsPersonID -> Application.People.PersonID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_AccountsPersonID_Application_People
FOREIGN KEY (AccountsPersonID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PersonID);

-- Foreign Key: Sales.Invoices.SalespersonPersonID -> Application.People.PersonID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_SalespersonPersonID_Application_People
FOREIGN KEY (SalespersonPersonID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PersonID);

-- Foreign Key: Sales.Invoices.PackedByPersonID -> Application.People.PersonID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICES
ADD CONSTRAINT FK_Sales_Invoices_PackedByPersonID_Application_People
FOREIGN KEY (PackedByPersonID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PEOPLE(PersonID);

SELECT * FROM SALES_INVOICES LIMIT 20

#### Cleaning WAREHOUSE_STOCKITEMTRANSACTIONS

In [ ]:
-- Create a clean table with the same structure as the raw table but we pull data from the raw table into this table (final table)
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS AS
SELECT
  CAST(CASE WHEN STOCKITEMTRANSACTIONID = 'NULL' THEN NULL ELSE STOCKITEMTRANSACTIONID END AS VARCHAR(38)) AS STOCKITEMTRANSACTIONID,
  CAST(CASE WHEN STOCKITEMID = 'NULL' THEN NULL ELSE STOCKITEMID END AS VARCHAR(38)) AS STOCKITEMID,
  CAST(CASE WHEN TRANSACTIONTYPEID = 'NULL' THEN NULL ELSE TRANSACTIONTYPEID END AS VARCHAR(38)) AS TRANSACTIONTYPEID,
  CAST(CASE WHEN CUSTOMERID = 'NULL' THEN NULL ELSE CUSTOMERID END AS VARCHAR(38)) AS CUSTOMERID,
  CAST(CASE WHEN INVOICEID = 'NULL' THEN NULL ELSE INVOICEID END AS VARCHAR(38)) AS INVOICEID,
  CAST(CASE WHEN SUPPLIERID = 'NULL' THEN NULL ELSE SUPPLIERID END AS VARCHAR(38)) AS SUPPLIERID,
  CAST(CASE WHEN PURCHASEORDERID = 'NULL' THEN NULL ELSE PURCHASEORDERID END AS VARCHAR(38)) AS PURCHASEORDERID,
  CAST(CASE WHEN TRANSACTIONOCCURREDWHEN = 'NULL' THEN NULL ELSE TRANSACTIONOCCURREDWHEN END AS VARCHAR(25)) AS TRANSACTIONOCCURREDWHEN,
  CAST(CASE WHEN QUANTITY = 'NULL' THEN NULL ELSE QUANTITY END AS VARCHAR(38)) AS QUANTITY
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS_RAW;


-- Null value checks for all columns
SELECT 
    SUM(CASE WHEN CUSTOMERID IS NULL THEN 1 END) AS NULL_COUNT_CUSTOMERID,
    SUM(CASE WHEN INVOICEID IS NULL THEN 1 END) AS NULL_COUNT_INVOICEID,
    SUM(CASE WHEN SUPPLIERID IS NULL THEN 1 END) AS NULL_COUNT_SUPPLIERID,
    SUM(CASE WHEN PURCHASEORDERID IS NULL THEN 1 END) AS NULL_COUNT_PURCHASEORDERID,
    SUM(CASE WHEN TRANSACTIONOCCURREDWHEN IS NULL THEN 1 END) AS NULL_COUNT_TRANSACTIONOCCURREDWHEN,
FROM 
    KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;
    

-- Null value checks for all columns (only numeric data type columns)
SELECT 
    COUNT(CASE WHEN STOCKITEMTRANSACTIONID IS NULL THEN 1 END) AS count_STOCKITEMTRANSACTIONID_NULL,
    COUNT(CASE WHEN STOCKITEMID IS NULL THEN 1 END) AS count_STOCKITEMID_NULL,
    COUNT(CASE WHEN TRANSACTIONTYPEID IS NULL THEN 1 END) AS count_TRANSACTIONTYPEID_NULL,
    COUNT(CASE WHEN QUANTITY IS NULL THEN 1 END) AS count_QUANTITY_NULL,
    COUNT(CASE WHEN CUSTOMERID IS NULL THEN 1 END) AS count_CUSTOMERID_NULL,
    COUNT(CASE WHEN INVOICEID IS NULL THEN 1 END) AS count_INVOICEID_NULL,
    COUNT(CASE WHEN SUPPLIERID IS NULL THEN 1 END) AS count_SUPPLIERID_NULL,
    COUNT(CASE WHEN PURCHASEORDERID IS NULL THEN 1 END) AS count_PURCHASEORDERID_NULL,
    COUNT(CASE WHEN TRANSACTIONOCCURREDWHEN IS NULL THEN 1 END) AS count_TRANSACTIONOCCURREDWHEN_NULL,
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

----------------------------------------------------------
-- STOCKITEMID
-----------------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN STOCKITEMID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET STOCKITEMID_NUM = TO_NUMBER(STOCKITEMID);

SELECT STOCKITEMID, STOCKITEMID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN STOCKITEMID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN STOCKITEMID_NUM TO STOCKITEMID;

------------------------------------------------------
-- STOCKITEMTRANSACTIONID
------------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN STOCKITEMTRANSACTIONID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET STOCKITEMTRANSACTIONID_NUM = TO_NUMBER(STOCKITEMTRANSACTIONID);

SELECT STOCKITEMTRANSACTIONID, STOCKITEMTRANSACTIONID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN STOCKITEMTRANSACTIONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN STOCKITEMTRANSACTIONID_NUM TO STOCKITEMTRANSACTIONID;

---------------------------------------------------------
-- TRANSACTIONTYPEID
----------------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN TRANSACTIONTYPEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET TRANSACTIONTYPEID_NUM = TO_NUMBER(TRANSACTIONTYPEID);

SELECT TRANSACTIONTYPEID, TRANSACTIONTYPEID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN TRANSACTIONTYPEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN TRANSACTIONTYPEID_NUM TO TRANSACTIONTYPEID;

--------------------------------------------------------
-- TRANSACTIONOCCURREDWHEN
-------------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN TRANSACTIONOCCURREDWHEN_DATE TIMESTAMP;

-- Change to date format for ORDERDATE column
UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET TRANSACTIONOCCURREDWHEN_DATE = TO_TIMESTAMP(TRANSACTIONOCCURREDWHEN, 'DD/MM/YYYY HH24:MI');

SELECT TRANSACTIONOCCURREDWHEN, TRANSACTIONOCCURREDWHEN_DATE
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN TRANSACTIONOCCURREDWHEN;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN TRANSACTIONOCCURREDWHEN_DATE TO TRANSACTIONOCCURREDWHEN;

--------------------------------------------
--CUSTOMERID
----------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN CUSTOMERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET CUSTOMERID_NUM = TO_NUMBER(CUSTOMERID);

SELECT CUSTOMERID, CUSTOMERID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN CUSTOMERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN CUSTOMERID_NUM TO CUSTOMERID;

--check the number of rows of customerid = 832(random customerid from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
WHERE CUSTOMERID = 832;


---------------------------------------------------------------
//INVOICEID
----------------------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN INVOICEID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET INVOICEID_NUM = TO_NUMBER(INVOICEID);

SELECT INVOICEID, INVOICEID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN INVOICEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN INVOICEID_NUM TO INVOICEID;

--check the number of rows of customerid = 103(random invoiceid from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
WHERE INVOICEID = 103;


----------------------------------------------------------
--SUPPLIERID
----------------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN SUPPLIERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET SUPPLIERID_NUM = TO_NUMBER(SUPPLIERID);

SELECT SUPPLIERID, SUPPLIERID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN SUPPLIERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN SUPPLIERID_NUM TO SUPPLIERID;

--check the number of rows of SUPPLIERID = 12(random SUPPLIERID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
WHERE SUPPLIERID = 12;


---------------------------------------------------
--PURCHASEORDERID
---------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN PURCHASEORDERID_NUM NUMBER(38,0);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET PURCHASEORDERID_NUM = TO_NUMBER(PURCHASEORDERID);

SELECT PURCHASEORDERID, PURCHASEORDERID_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN PURCHASEORDERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN PURCHASEORDERID_NUM TO PURCHASEORDERID;

--check the number of rows of PURCHASEORDERID = 109(random PURCHASEORDERID from data), cross checked with excel sheet
SELECT COUNT(*) AS row_count
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
WHERE PURCHASEORDERID = 109;


----------------------------------------------------
-- QUANTITY
-------------------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD COLUMN QUANTITY_NUM DECIMAL(18, 3);

UPDATE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
SET QUANTITY_NUM = TO_DECIMAL(QUANTITY, 18, 3);

SELECT QUANTITY, QUANTITY_NUM
FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
DROP COLUMN QUANTITY;

ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
RENAME COLUMN QUANTITY_NUM TO QUANTITY;

SELECT * FROM KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS LIMIT 10;


-- primary key
ALTER TABLE KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMTRANSACTIONS
ADD CONSTRAINT PK_WAREHOUSE_STOCKITEMTRANSACTIONS_STOCKITEMID
PRIMARY KEY (STOCKITEMID);


-- null error handling for primary key (increment 1 to impute null value primary keys from the previous record)
WITH CTE AS (
    SELECT 
        StockItemTransactionID,
        StockItemID,
        TransactionTypeID,
        CustomerID,
        InvoiceID,
        SupplierID,
        PurchaseOrderID,
        TransactionOccurredWhen,
        Quantity,
        LAG(StockItemTransactionID) OVER (ORDER BY StockItemTransactionID) AS prev_transactionid,
        ROW_NUMBER() OVER (ORDER BY StockItemTransactionID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.Warehouse_StockItemTransactions
)
SELECT 
    StockItemID,
    TransactionTypeID,
    CustomerID,
    InvoiceID,
    SupplierID,
    PurchaseOrderID,
    TransactionOccurredWhen,
    Quantity,
    CASE
        WHEN StockItemTransactionID IS NULL THEN prev_transactionid + 1
        ELSE StockItemTransactionID
    END AS StockItemTransactionID
FROM CTE
ORDER BY row_num;


-- done foreign keys

-- Foreign key for CustomerID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItemTransactions
ADD CONSTRAINT FK_Warehouse_StockItemTransactions_CustomerID_Sales_Customers
FOREIGN KEY (CustomerID)
REFERENCES KN_LOGISTICS.SNOWSQL.Sales_Customers (CustomerID);

-- Foreign key for PurchaseOrderID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItemTransactions
ADD CONSTRAINT FK_Warehouse_StockItemTransactions_PurchaseOrderID_Purchasing_PurchaseOrders
FOREIGN KEY (PurchaseOrderID)
REFERENCES KN_LOGISTICS.SNOWSQL.Purchasing_PurchaseOrders (PurchaseOrderID);

-- Foreign key for StockItemID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItemTransactions
ADD CONSTRAINT FK_Warehouse_StockItemTransactions_StockItemID_Warehouse_StockItems
FOREIGN KEY (StockItemID)
REFERENCES KN_LOGISTICS.SNOWSQL.Warehouse_StockItems (StockItemID);

-- Foreign key for SupplierID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItemTransactions
ADD CONSTRAINT FK_Warehouse_StockItemTransactions_SupplierID_Purchasing_Suppliers
FOREIGN KEY (SupplierID)
REFERENCES KN_LOGISTICS.SNOWSQL.Purchasing_Suppliers (SupplierID);

-- Foreign key for InvoiceID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItemTransactions
ADD CONSTRAINT FK_Warehouse_StockItemTransactions_InvoiceID_Sales_Invoices
FOREIGN KEY (InvoiceID)
REFERENCES KN_LOGISTICS.SNOWSQL.Sales_Invoices (InvoiceID);

-- Foreign key for TransactionTypeID
ALTER TABLE KN_LOGISTICS.SNOWSQL.Warehouse_StockItemTransactions
ADD CONSTRAINT FK_Warehouse_StockItemTransactions_TransactionTypeID_Application_TransactionTypes
FOREIGN KEY (TransactionTypeID)
REFERENCES KN_LOGISTICS.SNOWSQL.Application_TransactionTypes (TransactionTypeID);

#### Cleaning SALES_INVOICELINES

In [ ]:
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES AS
SELECT
  CAST(CASE WHEN DESCRIPTION = 'NULL' THEN NULL ELSE DESCRIPTION END AS VARCHAR(100)) AS DESCRIPTION,
  CAST(CASE WHEN INVOICELINEID = 'NULL' THEN NULL ELSE INVOICELINEID END AS VARCHAR(38)) AS INVOICELINEID,
  CAST(CASE WHEN INVOICEID = 'NULL' THEN NULL ELSE INVOICEID END AS VARCHAR(38)) AS INVOICEID,
  CAST(CASE WHEN STOCKITEMID = 'NULL' THEN NULL ELSE STOCKITEMID END AS VARCHAR(38)) AS STOCKITEMID,
  CAST(CASE WHEN PACKAGETYPEID = 'NULL' THEN NULL ELSE PACKAGETYPEID END AS VARCHAR(38)) AS PACKAGETYPEID,
  CAST(CASE WHEN QUANTITY = 'NULL' THEN NULL ELSE QUANTITY END AS VARCHAR(38)) AS QUANTITY,
  CAST(CASE WHEN UNITPRICE = 'NULL' THEN NULL ELSE UNITPRICE END AS VARCHAR(38)) AS UNITPRICE,
  CAST(CASE WHEN TAXAMOUNT = 'NULL' THEN NULL ELSE TAXAMOUNT END AS VARCHAR(38)) AS TAXAMOUNT,
  CAST(CASE WHEN LINEPROFIT = 'NULL' THEN NULL ELSE LINEPROFIT END AS VARCHAR(38)) AS LINEPROFIT,
  CAST(CASE WHEN EXTENDEDPRICE = 'NULL' THEN NULL ELSE EXTENDEDPRICE END AS VARCHAR(38)) AS EXTENDEDPRICE,
  CAST(CASE WHEN TAXRATE = 'NULL' THEN NULL ELSE TAXRATE END AS VARCHAR(38)) AS TAXRATE
FROM KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES_RAW;


------------------------------------
--Converting of datatypes
------------------------------------
-- INVOICELINEID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN INVOICELINEID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET INVOICELINEID_NUM = CAST(INVOICELINEID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN INVOICELINEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN INVOICELINEID_NUM TO INVOICELINEID;

-- INVOICEID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN INVOICEID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET INVOICEID_NUM = CAST(INVOICEID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN INVOICEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN INVOICEID_NUM TO INVOICEID;

-- STOCKITEMID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN STOCKITEMID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET STOCKITEMID_NUM = CAST(STOCKITEMID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN STOCKITEMID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN STOCKITEMID_NUM TO STOCKITEMID;

-- PACKAGETYPEID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN PACKAGETYPEID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET PACKAGETYPEID_NUM = CAST(PACKAGETYPEID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN PACKAGETYPEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN PACKAGETYPEID_NUM TO PACKAGETYPEID;

-- UNITPRICE
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN UNITPRICE_DEC DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET UNITPRICE_DEC = CAST(UNITPRICE AS DECIMAL(18, 2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN UNITPRICE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN UNITPRICE_DEC TO UNITPRICE;

-- LINEPROFIT
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN LINEPROFIT_DEC DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET LINEPROFIT_DEC = CAST(LINEPROFIT AS DECIMAL(18, 2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN LINEPROFIT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN LINEPROFIT_DEC TO LINEPROFIT;

-- EXTENDEDPRICE
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN EXTENDEDPRICE_DEC DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET EXTENDEDPRICE_DEC = CAST(EXTENDEDPRICE AS DECIMAL(18, 2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN EXTENDEDPRICE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN EXTENDEDPRICE_DEC TO EXTENDEDPRICE;

-- TAXAMOUNT
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN TAXAMOUNT_DEC DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET TAXAMOUNT_DEC = CAST(TAXAMOUNT AS DECIMAL(18, 2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN TAXAMOUNT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN TAXAMOUNT_DEC TO TAXAMOUNT;

-- TAXRATE
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD COLUMN TAXRATE_DEC DECIMAL(18, 3);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
SET TAXRATE_DEC = CAST(TAXRATE AS DECIMAL(18, 3));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
DROP COLUMN TAXRATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
RENAME COLUMN TAXRATE_DEC TO TAXRATE;


-------------------------------------
--Adding primary key to table
-------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD CONSTRAINT PK_SALES_INVOICELINES_INVOICELINEID
PRIMARY KEY (INVOICELINEID);


----------------------------------------------------
--ERROR HANDLING
-----------------------------------------------------
WITH CTE AS (
    SELECT 
        INVOICELINEID,
        INVOICEID,
        STOCKITEMID,
        DESCRIPTION,
        PACKAGETYPEID,
        QUANTITY,
        UNITPRICE,
        TAXAMOUNT,
        LINEPROFIT,
        EXTENDEDPRICE,
        TAXRATE,
        LAG(INVOICELINEID) OVER (ORDER BY INVOICELINEID) AS prev_invoicelineid,
        ROW_NUMBER() OVER (ORDER BY INVOICELINEID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
)
SELECT 
    INVOICEID,
    STOCKITEMID,
    DESCRIPTION,
    PACKAGETYPEID,
    QUANTITY,
    UNITPRICE,
    TAXAMOUNT,
    LINEPROFIT,
    EXTENDEDPRICE,
    TAXRATE,
    CASE
        WHEN INVOICELINEID IS NULL THEN prev_invoicelineid + 1
        ELSE INVOICELINEID
    END AS INVOICELINEID
FROM CTE
ORDER BY row_num;


-----------------------------------------------
--Adding foreign keys in the table
-----------------------------------------------
-- Foreign Key: Sales.InvoiceLines.InvoiceID -> Sales.Invoices.InvoiceID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD CONSTRAINT FK_SALES_INVOICELINES_INVOICEID_SALES_INVOICES
FOREIGN KEY (InvoiceID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_INVOICES(InvoiceID);

-- Foreign Key: Sales.InvoiceLines.StockItemID -> Warehouse.StockItems.StockItemID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD CONSTRAINT FK_SALES_INVOICELINES_STOCKITEMID_WAREHOUSE_STOCKITEMS
FOREIGN KEY (StockItemID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_STOCKITEMS(StockItemID);

-- Foreign Key: Sales.InvoiceLines.PackageTypeID -> Warehouse.PackageTypes.PackageTypeID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_INVOICELINES
ADD CONSTRAINT FK_SALES_INVOICELINES_PACKAGETYPEID_WAREHOUSE_PACKAGETYPES
FOREIGN KEY (PackageTypeID)
REFERENCES KN_LOGISTICS.SNOWSQL.WAREHOUSE_PACKAGETYPES(PackageTypeID);



#### Cleaning SALES_CUSTOMERTRANSACTIONS

In [ ]:
CREATE OR REPLACE TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS AS
SELECT
  CAST(CASE WHEN CUSTOMERTRANSACTIONID = 'NULL' THEN NULL ELSE CUSTOMERTRANSACTIONID END AS VARCHAR(38)) AS CUSTOMERTRANSACTIONID,
  CAST(CASE WHEN CUSTOMERID = 'NULL' THEN NULL ELSE CUSTOMERID END AS VARCHAR(38)) AS CUSTOMERID,
  CAST(CASE WHEN TRANSACTIONTYPEID = 'NULL' THEN NULL ELSE TRANSACTIONTYPEID END AS VARCHAR(38)) AS TRANSACTIONTYPEID,
  CAST(CASE WHEN INVOICEID = 'NULL' THEN NULL ELSE INVOICEID END AS VARCHAR(38)) AS INVOICEID,
  CAST(CASE WHEN PAYMENTMETHODID = 'NULL' THEN NULL ELSE PAYMENTMETHODID END AS VARCHAR(38)) AS PAYMENTMETHODID,
  CAST(CASE WHEN TRANSACTIONDATE = 'NULL' THEN NULL ELSE TRANSACTIONDATE END AS VARCHAR(20)) AS TRANSACTIONDATE,
  CAST(CASE WHEN AMOUNTEXCLUDINGTAX = 'NULL' THEN NULL ELSE AMOUNTEXCLUDINGTAX END AS VARCHAR(38)) AS AMOUNTEXCLUDINGTAX,
  CAST(CASE WHEN TAXAMOUNT = 'NULL' THEN NULL ELSE TAXAMOUNT END AS VARCHAR(38)) AS TAXAMOUNT,
  CAST(CASE WHEN TRANSACTIONAMOUNT = 'NULL' THEN NULL ELSE TRANSACTIONAMOUNT END AS VARCHAR(38)) AS TRANSACTIONAMOUNT,
  CAST(CASE WHEN OUTSTANDINGBALANCE = 'NULL' THEN NULL ELSE OUTSTANDINGBALANCE END AS VARCHAR(38)) AS OUTSTANDINGBALANCE,
  CAST(CASE WHEN FINALIZATIONDATE = 'NULL' THEN NULL ELSE FINALIZATIONDATE END AS VARCHAR(20)) AS FINALIZATIONDATE,
  CAST(CASE WHEN ISFINALIZED = 'NULL' THEN NULL ELSE ISFINALIZED END AS VARCHAR(38)) AS ISFINALIZED
FROM KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS_RAW;

------------------------------------
--Converting of datatypes
------------------------------------
-- Convert CUSTOMERTRANSACTIONID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN CUSTOMERTRANSACTIONID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET CUSTOMERTRANSACTIONID_NUM = CAST(CUSTOMERTRANSACTIONID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN CUSTOMERTRANSACTIONID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN CUSTOMERTRANSACTIONID_NUM TO CUSTOMERTRANSACTIONID;

-- Convert CUSTOMERID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN CUSTOMERID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET CUSTOMERID_NUM = CAST(CUSTOMERID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN CUSTOMERID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN CUSTOMERID_NUM TO CUSTOMERID;

-- Convert TRANSACTIONTYPEID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN TRANSACTIONTYPEID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET TRANSACTIONTYPEID_NUM = CAST(TRANSACTIONTYPEID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN TRANSACTIONTYPEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN TRANSACTIONTYPEID_NUM TO TRANSACTIONTYPEID;

-- Convert INVOICEID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN INVOICEID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET INVOICEID_NUM = CAST(INVOICEID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN INVOICEID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN INVOICEID_NUM TO INVOICEID;

-- Convert PAYMENTMETHODID to NUMBER(38,0)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN PAYMENTMETHODID_NUM NUMBER(38, 0);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET PAYMENTMETHODID_NUM = CAST(PAYMENTMETHODID AS NUMBER(38, 0));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN PAYMENTMETHODID;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN PAYMENTMETHODID_NUM TO PAYMENTMETHODID;

-- Convert TRANSACTIONDATE to DATE
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN TRANSACTIONDATE_DATE DATE;

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET TRANSACTIONDATE_DATE = TO_DATE(TRANSACTIONDATE, 'DD/MM/YYYY');

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN TRANSACTIONDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN TRANSACTIONDATE_DATE TO TRANSACTIONDATE;

-- Convert AMOUNTEXCLUDINGTAX to DECIMAL(18,2)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN AMOUNTEXCLUDINGTAX_DEC DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET AMOUNTEXCLUDINGTAX_DEC = CAST(AMOUNTEXCLUDINGTAX AS DECIMAL(18, 2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN AMOUNTEXCLUDINGTAX;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN AMOUNTEXCLUDINGTAX_DEC TO AMOUNTEXCLUDINGTAX;

-- Convert TAXAMOUNT to DECIMAL(18,2)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN TAXAMOUNT_DEC DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET TAXAMOUNT_DEC = CAST(TAXAMOUNT AS DECIMAL(18, 2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN TAXAMOUNT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN TAXAMOUNT_DEC TO TAXAMOUNT;

-- Convert TRANSACTIONAMOUNT to DECIMAL(18,2)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN TRANSACTIONAMOUNT_DEC DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET TRANSACTIONAMOUNT_DEC = CAST(TRANSACTIONAMOUNT AS DECIMAL(18, 2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN TRANSACTIONAMOUNT;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN TRANSACTIONAMOUNT_DEC TO TRANSACTIONAMOUNT;

-- Convert OUTSTANDINGBALANCE to DECIMAL(18,2)
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN OUTSTANDINGBALANCE_DEC DECIMAL(18, 2);

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET OUTSTANDINGBALANCE_DEC = CAST(OUTSTANDINGBALANCE AS DECIMAL(18, 2));

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN OUTSTANDINGBALANCE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN OUTSTANDINGBALANCE_DEC TO OUTSTANDINGBALANCE;

-- Convert FINALIZATIONDATE to DATE
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN FINALIZATIONDATE_DATE DATE;

-- Step 2: Convert and populate the new column using TO_DATE
UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET FINALIZATIONDATE_DATE = TO_DATE(FINALIZATIONDATE, 'DD/MM/YYYY');

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN FINALIZATIONDATE;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN FINALIZATIONDATE_DATE TO FINALIZATIONDATE;

-- Convert ISFINALIZED to BOOLEAN
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD COLUMN ISFINALIZED_BOOL BOOLEAN;

UPDATE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
SET ISFINALIZED_BOOL = 
    CASE 
        WHEN ISFINALIZED = '1' THEN TRUE
        WHEN ISFINALIZED = '0' THEN FALSE
        ELSE NULL
    END;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
DROP COLUMN ISFINALIZED;

ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
RENAME COLUMN ISFINALIZED_BOOL TO ISFINALIZED;


---------------------------------------
--Adding primary key to table
---------------------------------------
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD CONSTRAINT PK_SALES_CUSTOMERTRANSACTIONS_CUSTOMERTRANSACTIONID
PRIMARY KEY (CUSTOMERTRANSACTIONID);


-----------------------------------------
--ERROR HANDLING
-----------------------------------------
WITH CTE AS (
    SELECT 
        CUSTOMERTRANSACTIONID,
        CUSTOMERID,
        TRANSACTIONTYPEID,
        INVOICEID,
        PAYMENTMETHODID,
        TRANSACTIONDATE,
        AMOUNTEXCLUDINGTAX,
        TAXAMOUNT,
        TRANSACTIONAMOUNT,
        OUTSTANDINGBALANCE,
        FINALIZATIONDATE,
        ISFINALIZED,
        LAG(CUSTOMERTRANSACTIONID) OVER (ORDER BY CUSTOMERTRANSACTIONID) AS prev_customertransactionid,
        ROW_NUMBER() OVER (ORDER BY CUSTOMERTRANSACTIONID) AS row_num
    FROM KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
)
SELECT 
    CUSTOMERID,
    TRANSACTIONTYPEID,
    INVOICEID,
    PAYMENTMETHODID,
    TRANSACTIONDATE,
    AMOUNTEXCLUDINGTAX,
    TAXAMOUNT,
    TRANSACTIONAMOUNT,
    OUTSTANDINGBALANCE,
    FINALIZATIONDATE,
    ISFINALIZED,
    CASE
        WHEN CUSTOMERTRANSACTIONID IS NULL THEN prev_customertransactionid + 1
        ELSE CUSTOMERTRANSACTIONID
    END AS CUSTOMERTRANSACTIONID
FROM CTE
ORDER BY row_num;

-------------------------------------
--Adding of foreign keys
-------------------------------------
-- Add Foreign Key: CUSTOMERID -> Sales.Customers.CustomerID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD CONSTRAINT FK_SALES_CUSTOMERTRANSACTIONS_CUSTOMERID_SALES_CUSTOMERS
FOREIGN KEY (CustomerID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERS(CustomerID);

-- Add Foreign Key: TRANSACTIONTYPEID -> Application.TransactionTypes.TransactionTypeID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD CONSTRAINT FK_SALES_CUSTOMERTRANSACTIONS_TRANSACTIONTYPEID_APPLICATION_TRANSACTIONTYPES
FOREIGN KEY (TransactionTypeID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_TRANSACTIONTYPES(TransactionTypeID);

-- Add Foreign Key: INVOICEID -> Sales.Invoices.InvoiceID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD CONSTRAINT FK_SALES_CUSTOMERTRANSACTIONS_INVOICEID_SALES_INVOICES
FOREIGN KEY (InvoiceID)
REFERENCES KN_LOGISTICS.SNOWSQL.SALES_INVOICES(InvoiceID);

-- Add Foreign Key: PAYMENTMETHODID -> Application.PaymentMethods.PaymentMethodID
ALTER TABLE KN_LOGISTICS.SNOWSQL.SALES_CUSTOMERTRANSACTIONS
ADD CONSTRAINT FK_SALES_CUSTOMERTRANSACTIONS_PAYMENTMETHODID_APPLICATION_PAYMENTMETHODS
FOREIGN KEY (PaymentMethodID)
REFERENCES KN_LOGISTICS.SNOWSQL.APPLICATION_PAYMENTMETHODS(PaymentMethodID);